In [168]:
# from __future__ import absolute_import, division, print_function, unicode_literals
#https://blog.cambridgespark.com/robust-one-hot-encoding-in-python-3e29bfcec77e
import os
import numpy as np
#import tensorflow as tf
from sklearn import preprocessing

from tensorflow.keras import layers
import pandas as pd
pd.set_option('display.max_columns', 500)
#import tensorflow_probability as tfp
#import matplotlib.pyplot as plt

In [2]:
goldstandard = pd.read_csv("goldstandard.csv", sep=',')
goldstandard.head()
#goldstandard.status.value_counts()

,person_id,status
0,0,0.0
1,1,0.0
2,2,0.0
3,3,1.0
4,4,0.0


0.0 ->    8206

1.0 ->    928

In [3]:
person = pd.read_csv("person.csv", sep=',')
person.head()

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,0,8532,1961,5,19,1961-05-19,8527,38003564,2694,NaN,NaN,NaN,F,0,NaN,30,NaN,29
1,1,8507,1978,9,6,1978-09-06,8516,38003564,6495,NaN,NaN,NaN,M,0,NaN,13,NaN,29
2,2,8532,1940,9,26,1940-09-26,8552,38003564,7023,NaN,NaN,NaN,F,0,NaN,8,NaN,27
3,3,8532,1990,9,28,1990-09-28,8516,38003564,1143,NaN,NaN,NaN,F,0,NaN,13,NaN,29
4,4,8507,2019,10,30,2019-10-30,8516,38003564,7696,NaN,NaN,NaN,M,0,NaN,13,NaN,29


In [4]:
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   person_id                    9134 non-null   int64  
 1   gender_concept_id            9134 non-null   int64  
 2   year_of_birth                9134 non-null   int64  
 3   month_of_birth               9134 non-null   int64  
 4   day_of_birth                 9134 non-null   int64  
 5   birth_datetime               9134 non-null   object 
 6   race_concept_id              9134 non-null   int64  
 7   ethnicity_concept_id         9134 non-null   int64  
 8   location_id                  9134 non-null   int64  
 9   provider_id                  0 non-null      float64
 10  care_site_id                 0 non-null      float64
 11  person_source_value          0 non-null      float64
 12  gender_source_value          9134 non-null   object 
 13  gender_source_conc

In [5]:
person['age'] = 2020 - person['year_of_birth']

In [6]:
person['ethnicity_concept_id'].value_counts()

38003564    7392
38003563    1742
Name: ethnicity_concept_id, dtype: int64

In [7]:
person_status = pd.merge(person,
                        goldstandard[['person_id', 'status']],
                        on='person_id')

In [8]:
person_status.groupby('ethnicity_concept_id')['status'].value_counts()

ethnicity_concept_id  status
38003563              0.0       1571
                      1.0        171
38003564              0.0       6635
                      1.0        757
Name: status, dtype: int64

In [9]:
person_status.groupby('race_concept_id')['status'].value_counts()

race_concept_id  status
8515             0.0       1461
                 1.0        166
8516             0.0       1398
                 1.0        165
8527             0.0       3068
                 1.0        347
8552             0.0       2279
                 1.0        250
Name: status, dtype: int64

In [10]:
person.race_concept_id.value_counts()

8527    3415
8552    2529
8515    1627
8516    1563
Name: race_concept_id, dtype: int64

In [11]:
person_subset = person_status[["person_id","gender_source_value", "age", "ethnicity_concept_id","race_concept_id", "status"]]
#person_goldstandard = person_subset.merge(goldstandard, how = 'inner', on="person_id")
person_subset.head()

,person_id,gender_source_value,age,ethnicity_concept_id,race_concept_id,status
0,0,F,59,38003564,8527,0.0
1,1,M,42,38003564,8516,0.0
2,2,F,80,38003564,8552,0.0
3,3,F,30,38003564,8516,1.0
4,4,M,1,38003564,8516,0.0


In [12]:
person_onehot_columns = ["gender_source_value"]
onehot_columns_prefix = ["gender_source_value"]

person_gender = pd.get_dummies(person_subset,prefix=onehot_columns_prefix, columns=person_onehot_columns)
person_gender.shape

#measurement_subset

(9134, 7)

In [13]:
person_onehot_columns = ["race_concept_id"]
onehot_columns_prefix = ["race_concept_id"]

person_race = pd.get_dummies(person_gender,prefix=onehot_columns_prefix, columns=person_onehot_columns)
person_race.shape

(9134, 10)

In [14]:
person_onehot_columns = ["ethnicity_concept_id"]
onehot_columns_prefix = ["ethnicity_concept_id"]

person_ethnicity = pd.get_dummies(person_race,prefix=onehot_columns_prefix, columns=person_onehot_columns)
person_ethnicity.shape

(9134, 11)

In [15]:
person_ethnicity.head()

,person_id,age,status,gender_source_value_F,gender_source_value_M,race_concept_id_8515,race_concept_id_8516,race_concept_id_8527,race_concept_id_8552,ethnicity_concept_id_38003563,ethnicity_concept_id_38003564
0,0,59,0.0,1,0,0,0,1,0,0,1
1,1,42,0.0,0,1,0,1,0,0,0,1
2,2,80,0.0,1,0,0,0,0,1,0,1
3,3,30,1.0,1,0,0,1,0,0,0,1
4,4,1,0.0,0,1,0,1,0,0,0,1


In [16]:
person_gender.ethnicity_concept_id.value_counts()

38003564    7392
38003563    1742
Name: ethnicity_concept_id, dtype: int64

In [17]:
person_ethnicity.head()

,person_id,age,status,gender_source_value_F,gender_source_value_M,race_concept_id_8515,race_concept_id_8516,race_concept_id_8527,race_concept_id_8552,ethnicity_concept_id_38003563,ethnicity_concept_id_38003564
0,0,59,0.0,1,0,0,0,1,0,0,1
1,1,42,0.0,0,1,0,1,0,0,0,1
2,2,80,0.0,1,0,0,0,0,1,0,1
3,3,30,1.0,1,0,0,1,0,0,0,1
4,4,1,0.0,0,1,0,1,0,0,0,1


In [18]:
procedure_occurrence = pd.read_csv("procedure_occurrence.csv", sep=',')
procedure_occurrence.head()

,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value
0,1,7846,2735484,2018-04-14,2018-04-14,257,NaN,NaN,NaN,NaN,NaN,NaN,2735484,NaN
1,2,3872,2779572,2017-04-08,2017-04-08,257,NaN,NaN,NaN,NaN,NaN,NaN,2779572,NaN
2,3,1439,2007634,2011-08-29,2011-08-29,257,NaN,NaN,NaN,NaN,NaN,NaN,2007634,NaN
3,4,4547,2778899,2010-08-02,2010-08-02,257,NaN,NaN,NaN,NaN,NaN,NaN,2778899,NaN
4,5,8877,2616270,2015-11-07,2015-11-07,32468,NaN,NaN,NaN,NaN,NaN,NaN,2616270,NaN


In [19]:
procedure_occurrence_subset = procedure_occurrence[["person_id","procedure_concept_id", "procedure_type_concept_id"]]


In [20]:
procedure_occurrence_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34769 entries, 0 to 34768
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   person_id                  34769 non-null  int64
 1   procedure_concept_id       34769 non-null  int64
 2   procedure_type_concept_id  34769 non-null  int64
dtypes: int64(3)
memory usage: 815.0 KB


In [21]:
procedure_occurrence.person_id.unique().size


8725

In [22]:
procedure_occurrence.procedure_concept_id.unique().size


987

In [23]:
#df.pivot_table(values='val', index=df.index, columns='key', aggfunc='first')
#procedure_occurrence.pivot_table(values='val', index=df.index, columns='key', aggfunc='first')
procedure_occurrence.shape

(34769, 14)

In [24]:
procedure_occurrence.procedure_concept_id.dtypes
#procedure_occurrence.procedure_concept_id = procedure_occurrence.procedure_concept_id.astype('category')
#procedure_occurrence['procedure_concept_id'] = pd.Categorical(procedure_occurrence['procedure_concept_id'])
#procedure_occurrence['procedure_concept_id'] = pd.to_numeric(procedure_occurrence['procedure_concept_id'])

dtype('int64')

In [25]:
procedure_onehot_columns = ["procedure_concept_id"]
processed_procedure = pd.get_dummies(procedure_occurrence_subset,prefix='procedure_concept_id', columns=procedure_onehot_columns)
processed_procedure.shape

(34769, 989)

In [26]:
processed_procedure.head()

,person_id,procedure_type_concept_id,procedure_concept_id_1314318,procedure_concept_id_1314319,procedure_concept_id_1314321,procedure_concept_id_1314331,procedure_concept_id_1531630,procedure_concept_id_1531631,procedure_concept_id_2000015,procedure_concept_id_2000016,procedure_concept_id_2000018,procedure_concept_id_2000019,procedure_concept_id_2000025,procedure_concept_id_2000035,procedure_concept_id_2000037,procedure_concept_id_2000038,procedure_concept_id_2000042,procedure_concept_id_2000073,procedure_concept_id_2000127,procedure_concept_id_2000136,procedure_concept_id_2000140,procedure_concept_id_2000167,procedure_concept_id_2000189,procedure_concept_id_2000191,procedure_concept_id_2000219,procedure_concept_id_2000236,procedure_concept_id_2000282,procedure_concept_id_2000382,procedure_concept_id_2000591,procedure_concept_id_2000663,procedure_concept_id_2000839,procedure_concept_id_2001047,procedure_concept_id_2001079,procedure_concept_id_2001080,procedure_concept_id_2001108,procedure_concept_id_2001118,procedure_concept_id_2001121,procedure_concept_id_2001157,procedure_concept_id_2001275,procedure_concept_id_2001292,procedure_concept_id_2001298,procedure_concept_id_2001318,procedure_concept_id_2001353,procedure_concept_id_2001396,procedure_concept_id_2001409,procedure_concept_id_2001421,procedure_concept_id_2001506,procedure_concept_id_2001541,procedure_concept_id_2001549,procedure_concept_id_2001551,procedure_concept_id_2001581,procedure_concept_id_2001996,procedure_concept_id_2002085,procedure_concept_id_2002174,procedure_concept_id_2002222,procedure_concept_id_2002264,procedure_concept_id_2002307,procedure_concept_id_2002311,procedure_concept_id_2002363,procedure_concept_id_2002364,procedure_concept_id_2002442,procedure_concept_id_2002449,procedure_concept_id_2002585,procedure_concept_id_2002622,procedure_concept_id_2002643,procedure_concept_id_2002685,procedure_concept_id_2002747,procedure_concept_id_2003066,procedure_concept_id_2003149,procedure_concept_id_2003166,procedure_concept_id_2003287,procedure_concept_id_2003406,procedure_concept_id_2003445,procedure_concept_id_2003447,procedure_concept_id_2003450,procedure_concept_id_2003462,procedure_concept_id_2003463,procedure_concept_id_2003467,procedure_concept_id_2003510,procedure_concept_id_2003524,procedure_concept_id_2003626,procedure_concept_id_2003782,procedure_concept_id_2003783,procedure_concept_id_2003789,procedure_concept_id_2004263,procedure_concept_id_2004406,procedure_concept_id_2004449,procedure_concept_id_2004491,procedure_concept_id_2004586,procedure_concept_id_2004670,procedure_concept_id_2004746,procedure_concept_id_2004747,procedure_concept_id_2004765,procedure_concept_id_2004786,procedure_concept_id_2004811,procedure_concept_id_2004844,procedure_concept_id_2004870,procedure_concept_id_2004925,procedure_concept_id_2004952,procedure_concept_id_2004955,procedure_concept_id_2005117,procedure_concept_id_2005138,procedure_concept_id_2005141,procedure_concept_id_2005221,procedure_concept_id_2005232,procedure_concept_id_2005313,procedure_concept_id_2005318,procedure_concept_id_2005332,procedure_concept_id_2005334,procedure_concept_id_2005416,procedure_concept_id_2005476,procedure_concept_id_2005479,procedure_concept_id_2005480,procedure_concept_id_2005641,procedure_concept_id_2005748,procedure_concept_id_2005763,procedure_concept_id_2005812,procedure_concept_id_2005813,procedure_concept_id_2005817,procedure_concept_id_2005818,procedure_concept_id_2005868,procedure_concept_id_2005886,procedure_concept_id_2005888,procedure_concept_id_2005911,procedure_concept_id_2005922,procedure_concept_id_2005931,procedure_concept_id_2005944,procedure_concept_id_2005962,procedure_concept_id_2006004,procedure_concept_id_2006108,procedure_concept_id_2006126,procedure_concept_id_2006131,procedure_concept_id_2006147,procedure_concept_id_2006177,procedure_concept_id_2006180,procedure_concept_id_2006301,procedure_concept_id_2006394,procedure_concept_id_2006479,procedure_concept_i

In [27]:
procedure_onehot_columns = ["procedure_type_concept_id"]
processed_procedure_final = pd.get_dummies(processed_procedure,prefix='procedure_type_concept_id', columns=procedure_onehot_columns)
processed_procedure_final.shape

(34769, 990)

In [28]:
processed_procedure.procedure_type_concept_id.value_counts()

257      33214
32468     1555
Name: procedure_type_concept_id, dtype: int64

In [29]:
processed_procedure_grouped = processed_procedure_final.groupby('person_id').agg(sum)
#processed_condition_final['person_id'] = processed_condition_final.index
processed_procedure_grouped.reset_index(level=0, inplace=True)


processed_procedure_grouped.shape

(8725, 990)

In [30]:
processed_procedure_grouped.head()

,person_id,procedure_concept_id_1314318,procedure_concept_id_1314319,procedure_concept_id_1314321,procedure_concept_id_1314331,procedure_concept_id_1531630,procedure_concept_id_1531631,procedure_concept_id_2000015,procedure_concept_id_2000016,procedure_concept_id_2000018,procedure_concept_id_2000019,procedure_concept_id_2000025,procedure_concept_id_2000035,procedure_concept_id_2000037,procedure_concept_id_2000038,procedure_concept_id_2000042,procedure_concept_id_2000073,procedure_concept_id_2000127,procedure_concept_id_2000136,procedure_concept_id_2000140,procedure_concept_id_2000167,procedure_concept_id_2000189,procedure_concept_id_2000191,procedure_concept_id_2000219,procedure_concept_id_2000236,procedure_concept_id_2000282,procedure_concept_id_2000382,procedure_concept_id_2000591,procedure_concept_id_2000663,procedure_concept_id_2000839,procedure_concept_id_2001047,procedure_concept_id_2001079,procedure_concept_id_2001080,procedure_concept_id_2001108,procedure_concept_id_2001118,procedure_concept_id_2001121,procedure_concept_id_2001157,procedure_concept_id_2001275,procedure_concept_id_2001292,procedure_concept_id_2001298,procedure_concept_id_2001318,procedure_concept_id_2001353,procedure_concept_id_2001396,procedure_concept_id_2001409,procedure_concept_id_2001421,procedure_concept_id_2001506,procedure_concept_id_2001541,procedure_concept_id_2001549,procedure_concept_id_2001551,procedure_concept_id_2001581,procedure_concept_id_2001996,procedure_concept_id_2002085,procedure_concept_id_2002174,procedure_concept_id_2002222,procedure_concept_id_2002264,procedure_concept_id_2002307,procedure_concept_id_2002311,procedure_concept_id_2002363,procedure_concept_id_2002364,procedure_concept_id_2002442,procedure_concept_id_2002449,procedure_concept_id_2002585,procedure_concept_id_2002622,procedure_concept_id_2002643,procedure_concept_id_2002685,procedure_concept_id_2002747,procedure_concept_id_2003066,procedure_concept_id_2003149,procedure_concept_id_2003166,procedure_concept_id_2003287,procedure_concept_id_2003406,procedure_concept_id_2003445,procedure_concept_id_2003447,procedure_concept_id_2003450,procedure_concept_id_2003462,procedure_concept_id_2003463,procedure_concept_id_2003467,procedure_concept_id_2003510,procedure_concept_id_2003524,procedure_concept_id_2003626,procedure_concept_id_2003782,procedure_concept_id_2003783,procedure_concept_id_2003789,procedure_concept_id_2004263,procedure_concept_id_2004406,procedure_concept_id_2004449,procedure_concept_id_2004491,procedure_concept_id_2004586,procedure_concept_id_2004670,procedure_concept_id_2004746,procedure_concept_id_2004747,procedure_concept_id_2004765,procedure_concept_id_2004786,procedure_concept_id_2004811,procedure_concept_id_2004844,procedure_concept_id_2004870,procedure_concept_id_2004925,procedure_concept_id_2004952,procedure_concept_id_2004955,procedure_concept_id_2005117,procedure_concept_id_2005138,procedure_concept_id_2005141,procedure_concept_id_2005221,procedure_concept_id_2005232,procedure_concept_id_2005313,procedure_concept_id_2005318,procedure_concept_id_2005332,procedure_concept_id_2005334,procedure_concept_id_2005416,procedure_concept_id_2005476,procedure_concept_id_2005479,procedure_concept_id_2005480,procedure_concept_id_2005641,procedure_concept_id_2005748,procedure_concept_id_2005763,procedure_concept_id_2005812,procedure_concept_id_2005813,procedure_concept_id_2005817,procedure_concept_id_2005818,procedure_concept_id_2005868,procedure_concept_id_2005886,procedure_concept_id_2005888,procedure_concept_id_2005911,procedure_concept_id_2005922,procedure_concept_id_2005931,procedure_concept_id_2005944,procedure_concept_id_2005962,procedure_concept_id_2006004,procedure_concept_id_2006108,procedure_concept_id_2006126,procedure_concept_id_2006131,procedure_concept_id_2006147,procedure_concept_id_2006177,procedure_concept_id_2006180,procedure_concept_id_2006301,procedure_concept_id_2006394,procedure_concept_id_2006479,procedure_concept_id_2006513,procedure_concep

In [31]:
visit_occurrence = pd.read_csv("visit_occurrence.csv", sep=',')
visit_occurrence.head()

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id
0,NaN,8855,5083,2014-11-25,2014-11-25 15:10:00,2014-12-05,2014-12-05 15:10:00,44818518,NaN,NaN,NaN,5083,32209,NaN,32209,NaN,NaN
1,NaN,5468,9201,2012-04-11,2012-04-11 15:10:00,2012-04-19,2012-04-19 15:10:00,44818518,NaN,NaN,NaN,9201,32209,NaN,32209,NaN,NaN
2,NaN,3311,9201,2011-04-14,2011-04-14 15:10:00,2011-04-18,2011-04-18 15:10:00,44818518,NaN,NaN,NaN,9201,32209,NaN,32209,NaN,NaN
3,NaN,7368,581479,2012-08-05,2012-08-05 15:10:00,2012-08-14,2012-08-14 15:10:00,44818518,NaN,NaN,NaN,581479,32209,NaN,32209,NaN,NaN
4,NaN,7338,581479,2001-09-06,2001-09-06 15:10:00,2001-09-14,2001-09-14 15:10:00,44818518,NaN,NaN,NaN,581479,32209,NaN,32209,NaN,NaN


In [32]:
visit_occurrence.visit_type_concept_id.value_counts()

44818518    739399
Name: visit_type_concept_id, dtype: int64

In [33]:
visit_occurrence.visit_concept_id.value_counts()

5083        106066
581479      106016
9202        105763
9203        105630
44790889    105544
32037       105339
9201        105041
Name: visit_concept_id, dtype: int64

In [34]:
visit_occurrence.visit_source_concept_id.value_counts()

5083        106066
581479      106016
9202        105763
9203        105630
44790889    105544
32037       105339
9201        105041
Name: visit_source_concept_id, dtype: int64

In [35]:
visit_occurrence.person_id.unique().size


9119

In [36]:
visit_occurrence_subset = visit_occurrence[["person_id","visit_concept_id"]]


In [37]:
visit_onehot_columns = ["visit_concept_id"]
processed_visit = pd.get_dummies(visit_occurrence_subset,prefix='visit_concept_id', columns=visit_onehot_columns)
processed_visit.head()

,person_id,visit_concept_id_5083,visit_concept_id_9201,visit_concept_id_9202,visit_concept_id_9203,visit_concept_id_32037,visit_concept_id_581479,visit_concept_id_44790889
0,8855,1,0,0,0,0,0,0
1,5468,0,1,0,0,0,0,0
2,3311,0,1,0,0,0,0,0
3,7368,0,0,0,0,0,1,0
4,7338,0,0,0,0,0,1,0


In [38]:
processed_visit.shape

(739399, 8)

In [39]:
processed_visit_final = processed_visit.groupby('person_id').agg(sum)
#processed_condition_final['person_id'] = processed_condition_final.index
processed_visit_final.reset_index(level=0, inplace=True)


processed_visit_final.shape

(9119, 8)

In [40]:
processed_visit_final.head()

,person_id,visit_concept_id_5083,visit_concept_id_9201,visit_concept_id_9202,visit_concept_id_9203,visit_concept_id_32037,visit_concept_id_581479,visit_concept_id_44790889
0,0,13,12,10,14,12,14,16
1,1,15,11,12,14,12,8,11
2,2,11,12,11,9,18,12,14
3,3,20,13,9,11,11,12,12
4,4,0,0,0,0,1,0,0


In [41]:
observation = pd.read_csv("observation.csv", sep=',')
observation.head()

,observation_id,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,unit_concept_id,provider_id,visit_occurrence_id,visit_detail_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value
0,1,6497,1009229,2011-04-18,2011-04-18 09:58:00,3028553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1009229,NaN,NaN
1,2,1241,4196147,2017-01-14,2017-01-14 09:58:00,3028553,98.0,NaN,NaN,NaN,8554.0,NaN,NaN,NaN,NaN,4196147,%,NaN
2,3,2713,37208405,2016-10-05,2016-10-05 09:58:00,38000280,NaN,Never,763692.0,NaN,NaN,NaN,NaN,NaN,NaN,37208405,NaN,NaN
3,4,969,37208405,2016-04-09,2016-04-09 09:58:00,38000280,NaN,Never,763692.0,NaN,NaN,NaN,NaN,NaN,NaN,37208405,NaN,NaN
4,5,5800,4005823,2015-03-27,2015-03-27 09:58:00,38000280,NaN,Never,763692.0,NaN,NaN,NaN,NaN,NaN,NaN,4005823,NaN,NaN


In [42]:
observation.person_id.unique().size


9132

In [43]:
observation_subset = observation[["person_id","observation_concept_id", "observation_type_concept_id"]]
observation_subset.observation_concept_id.dtypes

dtype('int64')

In [44]:
observation_subset.observation_concept_id.value_counts()

37208405    1193054
4005823      298320
4196147      224065
1009229       74938
3003798       74907
Name: observation_concept_id, dtype: int64

In [45]:
observation_subset.observation_concept_id.unique()

array([ 1009229,  4196147, 37208405,  4005823,  3003798])

In [46]:
#observation_subset.observation_concept_id = observation_subset.observation_concept_id.astype('int')

In [47]:
observation_onehot_columns = ["observation_concept_id"]
processed_observation = pd.get_dummies(observation_subset,prefix='observation_concept_id', columns=observation_onehot_columns)
processed_observation.head()

,person_id,observation_type_concept_id,observation_concept_id_1009229,observation_concept_id_3003798,observation_concept_id_4005823,observation_concept_id_4196147,observation_concept_id_37208405
0,6497,3028553,1,0,0,0,0
1,1241,3028553,0,0,0,1,0
2,2713,38000280,0,0,0,0,1
3,969,38000280,0,0,0,0,1
4,5800,38000280,0,0,1,0,0


In [48]:
processed_observation.shape

(1865284, 7)

In [49]:
observation_onehot_columns = ["observation_type_concept_id"]
processed_observation_final = pd.get_dummies(processed_observation,prefix='observation_type_concept_id', columns=observation_onehot_columns)
processed_observation_final.shape

(1865284, 8)

In [50]:
processed_observation_grouped = processed_observation_final.groupby('person_id').agg(sum)
#processed_condition_final['person_id'] = processed_condition_final.index
processed_observation_grouped.reset_index(level=0, inplace=True)


processed_observation_grouped.shape

(9132, 8)

In [51]:
processed_observation_grouped.head()

,person_id,observation_concept_id_1009229,observation_concept_id_3003798,observation_concept_id_4005823,observation_concept_id_4196147,observation_concept_id_37208405,observation_type_concept_id_3028553,observation_type_concept_id_38000280
0,0,6,8,44,28,143,42,187
1,1,12,5,39,19,126,36,165
2,2,4,8,34,34,118,46,152
3,3,10,4,40,28,144,42,184
4,4,0,0,1,0,8,0,9


In [52]:
columns = processed_observation_grouped.columns
for col in columns:
    if col != "person_id":
        processed_observation_grouped[col] = processed_observation_grouped[col].apply(lambda x: 1 if x >0 else 0)


In [53]:
processed_observation.observation_type_concept_id.value_counts()

38000280    1491374
3028553      373910
Name: observation_type_concept_id, dtype: int64

In [54]:
processed_observation_grouped.head()

,person_id,observation_concept_id_1009229,observation_concept_id_3003798,observation_concept_id_4005823,observation_concept_id_4196147,observation_concept_id_37208405,observation_type_concept_id_3028553,observation_type_concept_id_38000280
0,0,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1
2,2,1,1,1,1,1,1,1
3,3,1,1,1,1,1,1,1
4,4,0,0,1,0,1,0,1


In [55]:
observation_period = pd.read_csv("observation_period.csv", sep=',')
observation_period.head()

,observation_period_id,person_id,observation_period_start_date,observation_period_end_date,period_type_concept_id
0,1,0,2001-07-14,2019-11-03,44814724
1,2,1,2001-08-31,2020-03-02,44814724
2,3,2,2001-07-14,2019-11-08,44814724
3,4,3,2001-06-17,2020-03-02,44814724
4,5,4,2019-12-29,2020-01-06,44814724


In [56]:
observation_period.period_type_concept_id.unique().size


1

In [57]:
condition_occurrence = pd.read_csv("condition_occurrence.csv", sep=',')
condition_occurrence.head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,1,4134,197672,2012-01-26,2012-01-26,2012-01-29,2012-01-29,32020,NaN,NaN,NaN,NaN,NaN,35211339,ENCOUNTER/ORDER SUMMARY,4033240
1,2,2503,4110961,2012-08-12,2012-08-12,2012-08-17,2012-08-17,32019,NaN,NaN,NaN,NaN,NaN,35207704,CHARGES,4230359
2,3,4216,192450,2014-05-21,2014-05-21,2014-05-26,2014-05-26,32019,NaN,NaN,NaN,NaN,NaN,45548960,BILLING,4230359
3,4,4603,80767,2017-12-06,2017-12-06,2017-12-12,2017-12-12,32019,NaN,NaN,NaN,NaN,NaN,35209357,BILLING,4230359
4,5,5538,441267,2018-06-16,2018-06-16,2018-06-22,2018-06-22,32020,NaN,NaN,NaN,NaN,NaN,35207084,ENCOUNTER/ORDER SUMMARY,4033240


In [58]:
condition_occurrence.person_id.unique().size


9133

In [59]:
condition_occurrence.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3357108 entries, 0 to 3357107
Data columns (total 16 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   condition_occurrence_id        int64  
 1   person_id                      int64  
 2   condition_concept_id           int64  
 3   condition_start_date           object 
 4   condition_start_datetime       object 
 5   condition_end_date             object 
 6   condition_end_datetime         object 
 7   condition_type_concept_id      int64  
 8   stop_reason                    float64
 9   provider_id                    float64
 10  visit_occurrence_id            float64
 11  visit_detail_id                float64
 12  condition_source_value         float64
 13  condition_source_concept_id    int64  
 14  condition_status_source_value  object 
 15  condition_status_concept_id    int64  
dtypes: float64(5), int64(6), object(5)
memory usage: 409.8+ MB


In [60]:
condition_occurrence['condition_source_concept_id'].value_counts()

35206146    7994
35225406    7890
35207793    7816
35225405    7807
45562087    6051
            ... 
35206320    1854
35207977    1853
45591008    1848
45544138    1847
35206137    1836
Name: condition_source_concept_id, Length: 1009, dtype: int64

In [61]:
condition_occurrence['condition_concept_id'].unique().size

864

In [62]:
condition_occurrence['condition_source_concept_id'].unique().size

1009

In [63]:
condition_occurrence['condition_status_concept_id'].value_counts()

4230359    1872060
4033240    1475281
4203942       9767
Name: condition_status_concept_id, dtype: int64

In [64]:
condition_occurrence['condition_type_concept_id'].value_counts()

32019       1920741
32020        802911
43542353     623689
42894222       9767
Name: condition_type_concept_id, dtype: int64

In [65]:
condition_occurrence['condition_status_source_value'].value_counts()

CHARGES                    976375
ENCOUNTER/ORDER SUMMARY    802911
BILLING                    717406
PROCEDURE                  359123
PRESCRIPTION               262663
DISCHARGE                  100118
APPOINTMENT                 46735
BILLING                     41067
FINAL                       25286
OTHER                       11808
ADMITTING                    9767
WORKING                      1946
ORDER                        1903
Name: condition_status_source_value, dtype: int64

In [66]:
#4356, 4901, 4729, 2386, 3929, 922
condition_occurrence.loc[condition_occurrence['person_id'].isin([4356, 4901, 4729, 2386, 3929, 922])].head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
92372,92373,922,4169954,2020-02-10,2020-02-10,2020-02-16,2020-02-16,32019,NaN,NaN,NaN,NaN,NaN,35209481,CHARGES,4230359
125942,125943,922,381551,2020-01-11,2020-01-11,2020-01-16,2020-01-16,32019,NaN,NaN,NaN,NaN,NaN,35207455,BILLING,4230359
270173,270174,4356,4336254,2020-03-11,2020-03-11,2020-03-17,2020-03-17,32019,NaN,NaN,NaN,NaN,NaN,35208585,CHARGES,4230359
295519,295520,2386,440360,2020-02-23,2020-02-23,2020-02-25,2020-02-25,32019,NaN,NaN,NaN,NaN,NaN,35207061,CHARGES,4230359
304198,304199,4729,137351,2020-01-03,2020-01-03,2020-01-06,2020-01-06,32019,NaN,NaN,NaN,NaN,NaN,45552995,CHARGES,4230359


In [67]:
condition_occurrence_subset = condition_occurrence[["person_id","condition_concept_id", "condition_source_concept_id","condition_status_source_value"]]


In [68]:
#encounter = condition_occurrence.loc[condition_occurrence['condition_status_source_value'].isin(['ENCOUNTER/ORDER SUMMARY'])]
ehr_line_df = condition_occurrence_subset.loc[condition_occurrence_subset['condition_status_source_value'].isin(['ENCOUNTER/ORDER SUMMARY'])]
ehr_line_df.person_id.unique().size
#encounter_personid = ehr_line_df.person_id.unique()
ehr_line_df.shape
condition_occurrence_subset = ehr_line_df

In [69]:
#condition_occurrence_subset = condition_occurrence_subset.drop_duplicates()
#condition_occurrence_subset.shape

In [70]:
del condition_occurrence_subset["condition_status_source_value"]

In [71]:
condition_occurrence_subset.shape

(802911, 3)

In [72]:
condition_onehot_columns = ["condition_concept_id"]
condition_onehot_columns_prefix = ["condition_concept_id"]
processed_condition_1 = pd.get_dummies(condition_occurrence_subset,prefix=condition_onehot_columns_prefix, columns=condition_onehot_columns)
processed_condition_1.head()

,person_id,condition_source_concept_id,condition_concept_id_24134,condition_concept_id_25297,condition_concept_id_26662,condition_concept_id_27674,condition_concept_id_29735,condition_concept_id_30437,condition_concept_id_31317,condition_concept_id_31821,condition_concept_id_31967,condition_concept_id_73754,condition_concept_id_73819,condition_concept_id_74635,condition_concept_id_75650,condition_concept_id_75860,condition_concept_id_75909,condition_concept_id_77074,condition_concept_id_77310,condition_concept_id_77635,condition_concept_id_77670,condition_concept_id_78232,condition_concept_id_78508,condition_concept_id_79061,condition_concept_id_79864,condition_concept_id_79908,condition_concept_id_80180,condition_concept_id_80502,condition_concept_id_80809,condition_concept_id_81902,condition_concept_id_81931,condition_concept_id_133141,condition_concept_id_134159,condition_concept_id_134460,condition_concept_id_134736,condition_concept_id_134898,condition_concept_id_135618,condition_concept_id_135777,condition_concept_id_135930,condition_concept_id_136773,condition_concept_id_136788,condition_concept_id_137053,condition_concept_id_137548,condition_concept_id_137809,condition_concept_id_137820,condition_concept_id_138384,condition_concept_id_138525,condition_concept_id_138825,condition_concept_id_140168,condition_concept_id_140214,condition_concept_id_140648,condition_concept_id_140673,condition_concept_id_140821,condition_concept_id_141095,condition_concept_id_141323,condition_concept_id_141663,condition_concept_id_141932,condition_concept_id_192450,condition_concept_id_193322,condition_concept_id_193326,condition_concept_id_193782,condition_concept_id_194081,condition_concept_id_194133,condition_concept_id_194696,condition_concept_id_195083,condition_concept_id_195771,condition_concept_id_195847,condition_concept_id_195867,condition_concept_id_196168,condition_concept_id_196360,condition_concept_id_196523,condition_concept_id_197320,condition_concept_id_197381,condition_concept_id_197494,condition_concept_id_197607,condition_concept_id_197672,condition_concept_id_197684,condition_concept_id_197925,condition_concept_id_197988,condition_concept_id_198263,condition_concept_id_198803,condition_concept_id_198964,condition_concept_id_199075,condition_concept_id_200174,condition_concept_id_200219,condition_concept_id_200452,condition_concept_id_200528,condition_concept_id_200843,condition_concept_id_200845,condition_concept_id_200962,condition_concept_id_201078,condition_concept_id_201254,condition_concept_id_201461,condition_concept_id_201618,condition_concept_id_201620,condition_concept_id_201826,condition_concept_id_254061,condition_concept_id_254320,condition_concept_id_254761,condition_concept_id_255573,condition_concept_id_255848,condition_concept_id_256439,condition_concept_id_256449,condition_concept_id_256451,condition_concept_id_257004,condition_concept_id_257007,condition_concept_id_257011,condition_concept_id_257012,condition_concept_id_257628,condition_concept_id_259848,condition_concept_id_260123,condition_concept_id_260139,condition_concept_id_261325,condition_concept_id_312337,condition_concept_id_312437,condition_concept_id_312998,condition_concept_id_313217,condition_concept_id_313459,condition_concept_id_314659,condition_concept_id_314665,condition_concept_id_314754,condition_concept_id_314962,condition_concept_id_315078,condition_concept_id_316139,condition_concept_id_318736,condition_concept_id_319041,condition_concept_id_319826,condition_concept_id_319835,condition_concept_id_320128,condition_concept_id_320536,condition_concept_id_320744,condition_concept_id_321052,condition_concept_id_321318,condition_concept_id_321319,condition_concept_id_321588,condition_concept_id_321596,condition_concept_id_372409,condition_concept_id_372448,condition_concept_id_373478,condition_concept_id_373503,condition_concept_id_374034,condition_concept_id_374375,condition_concept_id_374919,condition_concept_id_375545,condition_conc

In [73]:
processed_condition_1.shape

(802911, 370)

In [74]:
condition_occurrence_subset['condition_concept_id'].unique().size

368

In [75]:
processed_condition_1['condition_source_concept_id'].unique().size

411

In [76]:
condition_onehot_columns = ["condition_source_concept_id"]
condition_onehot_columns_prefix = ["condition_source_concept_id"]
processed_condition_2 = pd.get_dummies(processed_condition_1,prefix=condition_onehot_columns_prefix, columns=condition_onehot_columns)
processed_condition_2.head()

,person_id,condition_concept_id_24134,condition_concept_id_25297,condition_concept_id_26662,condition_concept_id_27674,condition_concept_id_29735,condition_concept_id_30437,condition_concept_id_31317,condition_concept_id_31821,condition_concept_id_31967,condition_concept_id_73754,condition_concept_id_73819,condition_concept_id_74635,condition_concept_id_75650,condition_concept_id_75860,condition_concept_id_75909,condition_concept_id_77074,condition_concept_id_77310,condition_concept_id_77635,condition_concept_id_77670,condition_concept_id_78232,condition_concept_id_78508,condition_concept_id_79061,condition_concept_id_79864,condition_concept_id_79908,condition_concept_id_80180,condition_concept_id_80502,condition_concept_id_80809,condition_concept_id_81902,condition_concept_id_81931,condition_concept_id_133141,condition_concept_id_134159,condition_concept_id_134460,condition_concept_id_134736,condition_concept_id_134898,condition_concept_id_135618,condition_concept_id_135777,condition_concept_id_135930,condition_concept_id_136773,condition_concept_id_136788,condition_concept_id_137053,condition_concept_id_137548,condition_concept_id_137809,condition_concept_id_137820,condition_concept_id_138384,condition_concept_id_138525,condition_concept_id_138825,condition_concept_id_140168,condition_concept_id_140214,condition_concept_id_140648,condition_concept_id_140673,condition_concept_id_140821,condition_concept_id_141095,condition_concept_id_141323,condition_concept_id_141663,condition_concept_id_141932,condition_concept_id_192450,condition_concept_id_193322,condition_concept_id_193326,condition_concept_id_193782,condition_concept_id_194081,condition_concept_id_194133,condition_concept_id_194696,condition_concept_id_195083,condition_concept_id_195771,condition_concept_id_195847,condition_concept_id_195867,condition_concept_id_196168,condition_concept_id_196360,condition_concept_id_196523,condition_concept_id_197320,condition_concept_id_197381,condition_concept_id_197494,condition_concept_id_197607,condition_concept_id_197672,condition_concept_id_197684,condition_concept_id_197925,condition_concept_id_197988,condition_concept_id_198263,condition_concept_id_198803,condition_concept_id_198964,condition_concept_id_199075,condition_concept_id_200174,condition_concept_id_200219,condition_concept_id_200452,condition_concept_id_200528,condition_concept_id_200843,condition_concept_id_200845,condition_concept_id_200962,condition_concept_id_201078,condition_concept_id_201254,condition_concept_id_201461,condition_concept_id_201618,condition_concept_id_201620,condition_concept_id_201826,condition_concept_id_254061,condition_concept_id_254320,condition_concept_id_254761,condition_concept_id_255573,condition_concept_id_255848,condition_concept_id_256439,condition_concept_id_256449,condition_concept_id_256451,condition_concept_id_257004,condition_concept_id_257007,condition_concept_id_257011,condition_concept_id_257012,condition_concept_id_257628,condition_concept_id_259848,condition_concept_id_260123,condition_concept_id_260139,condition_concept_id_261325,condition_concept_id_312337,condition_concept_id_312437,condition_concept_id_312998,condition_concept_id_313217,condition_concept_id_313459,condition_concept_id_314659,condition_concept_id_314665,condition_concept_id_314754,condition_concept_id_314962,condition_concept_id_315078,condition_concept_id_316139,condition_concept_id_318736,condition_concept_id_319041,condition_concept_id_319826,condition_concept_id_319835,condition_concept_id_320128,condition_concept_id_320536,condition_concept_id_320744,condition_concept_id_321052,condition_concept_id_321318,condition_concept_id_321319,condition_concept_id_321588,condition_concept_id_321596,condition_concept_id_372409,condition_concept_id_372448,condition_concept_id_373478,condition_concept_id_373503,condition_concept_id_374034,condition_concept_id_374375,condition_concept_id_374919,condition_concept_id_375545,condition_concept_id_376065,condition_conc

In [77]:
processed_condition_2.shape

(802911, 780)

In [78]:
#s = processed_condition_2.set_index('person_id').stack()
#df = s[s.eq(1)].unstack().fillna(0).astype(int).reset_index()
#df.shape()

In [79]:
processed_condition_final = processed_condition_2.groupby('person_id').agg(sum)
#processed_condition_final['person_id'] = processed_condition_final.index
processed_condition_final.reset_index(level=0, inplace=True)


In [80]:
processed_condition_final.shape

(9128, 780)

In [81]:
processed_condition_final.head()

,person_id,condition_concept_id_24134,condition_concept_id_25297,condition_concept_id_26662,condition_concept_id_27674,condition_concept_id_29735,condition_concept_id_30437,condition_concept_id_31317,condition_concept_id_31821,condition_concept_id_31967,condition_concept_id_73754,condition_concept_id_73819,condition_concept_id_74635,condition_concept_id_75650,condition_concept_id_75860,condition_concept_id_75909,condition_concept_id_77074,condition_concept_id_77310,condition_concept_id_77635,condition_concept_id_77670,condition_concept_id_78232,condition_concept_id_78508,condition_concept_id_79061,condition_concept_id_79864,condition_concept_id_79908,condition_concept_id_80180,condition_concept_id_80502,condition_concept_id_80809,condition_concept_id_81902,condition_concept_id_81931,condition_concept_id_133141,condition_concept_id_134159,condition_concept_id_134460,condition_concept_id_134736,condition_concept_id_134898,condition_concept_id_135618,condition_concept_id_135777,condition_concept_id_135930,condition_concept_id_136773,condition_concept_id_136788,condition_concept_id_137053,condition_concept_id_137548,condition_concept_id_137809,condition_concept_id_137820,condition_concept_id_138384,condition_concept_id_138525,condition_concept_id_138825,condition_concept_id_140168,condition_concept_id_140214,condition_concept_id_140648,condition_concept_id_140673,condition_concept_id_140821,condition_concept_id_141095,condition_concept_id_141323,condition_concept_id_141663,condition_concept_id_141932,condition_concept_id_192450,condition_concept_id_193322,condition_concept_id_193326,condition_concept_id_193782,condition_concept_id_194081,condition_concept_id_194133,condition_concept_id_194696,condition_concept_id_195083,condition_concept_id_195771,condition_concept_id_195847,condition_concept_id_195867,condition_concept_id_196168,condition_concept_id_196360,condition_concept_id_196523,condition_concept_id_197320,condition_concept_id_197381,condition_concept_id_197494,condition_concept_id_197607,condition_concept_id_197672,condition_concept_id_197684,condition_concept_id_197925,condition_concept_id_197988,condition_concept_id_198263,condition_concept_id_198803,condition_concept_id_198964,condition_concept_id_199075,condition_concept_id_200174,condition_concept_id_200219,condition_concept_id_200452,condition_concept_id_200528,condition_concept_id_200843,condition_concept_id_200845,condition_concept_id_200962,condition_concept_id_201078,condition_concept_id_201254,condition_concept_id_201461,condition_concept_id_201618,condition_concept_id_201620,condition_concept_id_201826,condition_concept_id_254061,condition_concept_id_254320,condition_concept_id_254761,condition_concept_id_255573,condition_concept_id_255848,condition_concept_id_256439,condition_concept_id_256449,condition_concept_id_256451,condition_concept_id_257004,condition_concept_id_257007,condition_concept_id_257011,condition_concept_id_257012,condition_concept_id_257628,condition_concept_id_259848,condition_concept_id_260123,condition_concept_id_260139,condition_concept_id_261325,condition_concept_id_312337,condition_concept_id_312437,condition_concept_id_312998,condition_concept_id_313217,condition_concept_id_313459,condition_concept_id_314659,condition_concept_id_314665,condition_concept_id_314754,condition_concept_id_314962,condition_concept_id_315078,condition_concept_id_316139,condition_concept_id_318736,condition_concept_id_319041,condition_concept_id_319826,condition_concept_id_319835,condition_concept_id_320128,condition_concept_id_320536,condition_concept_id_320744,condition_concept_id_321052,condition_concept_id_321318,condition_concept_id_321319,condition_concept_id_321588,condition_concept_id_321596,condition_concept_id_372409,condition_concept_id_372448,condition_concept_id_373478,condition_concept_id_373503,condition_concept_id_374034,condition_concept_id_374375,condition_concept_id_374919,condition_concept_id_375545,condition_concept_id_376065,condition_conc

In [82]:
columns = processed_condition_final.columns
for col in columns:
    print(col)


person_id
condition_concept_id_24134
condition_concept_id_25297
condition_concept_id_26662
condition_concept_id_27674
condition_concept_id_29735
condition_concept_id_30437
condition_concept_id_31317
condition_concept_id_31821
condition_concept_id_31967
condition_concept_id_73754
condition_concept_id_73819
condition_concept_id_74635
condition_concept_id_75650
condition_concept_id_75860
condition_concept_id_75909
condition_concept_id_77074
condition_concept_id_77310
condition_concept_id_77635
condition_concept_id_77670
condition_concept_id_78232
condition_concept_id_78508
condition_concept_id_79061
condition_concept_id_79864
condition_concept_id_79908
condition_concept_id_80180
condition_concept_id_80502
condition_concept_id_80809
condition_concept_id_81902
condition_concept_id_81931
condition_concept_id_133141
condition_concept_id_134159
condition_concept_id_134460
condition_concept_id_134736
condition_concept_id_134898
condition_concept_id_135618
condition_concept_id_135777
condition_c

In [83]:
p = set(person.person_id)
s = set(processed_condition_final.person_id)
#print(p)

print(p - s)

{4356, 4901, 4729, 2386, 3929, 922}


In [84]:
condition_era = pd.read_csv("condition_era.csv", sep=',')
condition_era.head()

,condition_era_id,person_id,condition_concept_id,condition_era_start_date,condition_era_end_date,condition_occurrence_count
0,1,0,24134,2017-01-01,2019-11-04,3
1,2,0,24974,2015-09-25,2015-09-28,1
2,3,0,26662,2012-11-11,2012-11-15,1
3,4,0,26942,2011-03-02,2011-03-07,1
4,5,0,29735,2010-01-30,2010-02-04,1


In [85]:
condition_era.shape

(2520147, 6)

In [86]:
condition_occurrence_count_data = condition_era.pivot_table(values='condition_occurrence_count', index='person_id', columns='condition_concept_id', fill_value=0, aggfunc='mean')
condition_occurrence_count_data.head()

condition_concept_id,22340,23325,24134,24818,24974,25297,26638,26662,26727,26942,27674,29735,30437,30683,30753,30978,31317,31509,31610,31821,31967,72266,72418,73754,73819,73840,74132,74582,74635,75004,75036,75576,75650,75860,75909,77074,77234,77310,77635,77670,78097,78110,78232,78234,78472,78508,79061,79864,79908,80045,80180,80502,80767,80809,80813,81250,81893,81902,81931,132706,132736,132797,132841,132853,133141,133169,133228,133283,133424,133438,133711,133729,133810,134159,134305,134438,134441,134442,134460,134603,134668,134736,134765,134898,135204,135496,135618,135777,135930,136057,136368,136497,136773,136788,137053,137351,137548,137682,137809,137820,137829,138102,138384,138387,138502,138525,138717,138825,138994,139099,139900,140168,140214,140352,140641,140648,140666,140673,140821,141095,141232,141323,141663,141816,141932,141941,141960,192359,192450,192568,192606,192671,192680,192854,192963,193138,193322,193326,193402,193518,193782,194081,194133,194175,194589,194684,194686,194696,194984,195007,195083,195321,195562,195575,195585,195596,195771,195847,195856,195867,196168,196236,196359,196360,196456,196463,196523,196925,197006,197236,197320,197381,197494,197500,197607,197610,197672,197675,197684,197806,197917,197925,197988,198075,198091,198263,198464,198683,198803,198964,198985,199065,199074,199075,199754,199837,199991,200051,200174,200219,200452,200527,200528,200762,200843,200845,200962,201061,201066,201078,201231,201254,201337,201340,201461,201519,201603,201606,201618,201620,201826,201900,201909,201916,253549,253796,254061,254320,254443,254591,254669,254761,255348,255573,255848,256439,256449,256451,256633,257004,257007,257011,257012,257628,257755,257907,258540,259153,259848,260123,...,4115169,4115170,4115171,4115367,4116166,4117695,4119786,4120090,4124693,4128329,4132434,4133224,4133599,4134120,4134121,4134455,4137430,4137468,4138617,4138760,4140164,4141360,4141454,4142875,4144111,4145825,4146087,4146209,4146581,4147411,4147466,4147829,4149353,4149904,4150062,4150125,4151763,4152351,4152384,4153359,4154088,4154168,4154290,4155909,4157454,4158910,4158911,4162253,4162860,4163710,4164151,4164898,4165998,4166125,4166126,4166231,4167085,4168212,4168213,4168217,4168222,4168700,4168796,4169095,4169954,4170554,4171394,4171917,4174262,4177206,4178114,4179823,4180628,4181965,4182210,4182562,4184976,4185471,4187218,4187355,4188155,4188331,4188598,4190771,4191597,4193704,4195085,4197819,4201387,4201388,4201745,4204199,4204991,4209423,4211231,4213271,4214376,4215402,4216771,4218813,4223659,4224926,4224940,4225273,4228361,4228802,4229440,4230641,4231770,4232697,4233565,4234997,4236312,4236484,4239381,4239938,4241530,4242669,4242816,4244078,4244438,4245252,4245842,4245975,4246451,4247238,4247719,4254485,4256761,4260546,4261842,4264693,4267582,4268911,4270024,4271013,4272240,4273250,4273391,4275359,4275423,4278669,4280354,4280726,4281109,4282096,4282316,4285717,4285898,4286201,4286497,4288544,4289309,4299417,4300092,4300704,4301699,4302954,4304484,4305376,4306267,4307820,4307956,4307981,4310235,4311499,4322024,4327889,4329041,4330445,4332246,4336254,4339214,35624756,35624757,36712821,36713112,36714126,36714994,36717114,37016193,37016768,37017431,37017432,37018196,37018854,37018875,37109056,37109843,37119138,37206607,37311319,37396726,40304526,40321716,40405599,40443308,40479573,40480893,40481089,40481901,40482241,40482893,40483107,40483172,40483287,40484532,40487528,42536696,42537730,42537741,42537742,42538117,42538119,42539502,42539698,42873170,43021974,43530652,43530691,43530807,43531003,43531028,43531561,44782429,44784217,44784631,44806773,44808984,45757468,45763549,45763750,45766207,45766714,45768812,45768910,45768963,45768964,45770830,45770892,45771045,46269757,46269816,46269889,46270513,46271022,46271075,46273463,46273487,46273652
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [87]:
condition_occurrence_count_data = condition_occurrence_count_data.add_prefix('condition_concept_id_')
condition_occurrence_count_data.reset_index(level=0, inplace=True)
condition_occurrence_count_data.columns


Index(['person_id', 'condition_concept_id_22340', 'condition_concept_id_23325',
       'condition_concept_id_24134', 'condition_concept_id_24818',
       'condition_concept_id_24974', 'condition_concept_id_25297',
       'condition_concept_id_26638', 'condition_concept_id_26662',
       'condition_concept_id_26727',
       ...
       'condition_concept_id_45771045', 'condition_concept_id_46269757',
       'condition_concept_id_46269816', 'condition_concept_id_46269889',
       'condition_concept_id_46270513', 'condition_concept_id_46271022',
       'condition_concept_id_46271075', 'condition_concept_id_46273463',
       'condition_concept_id_46273487', 'condition_concept_id_46273652'],
      dtype='object', name='condition_concept_id', length=865)

In [88]:
condition_occurrence_count_data.person_id.unique().size

9133

In [89]:
drug_era = pd.read_csv("drug_era.csv", sep=',')
drug_era.head()


,drug_era_id,person_id,drug_concept_id,drug_era_start_date,drug_era_end_date,drug_exposure_count,gap_days
0,1,0,535714,2019-06-20,2019-06-21,1,0
1,2,0,704601,2019-09-19,2019-09-19,1,0
2,3,0,704943,2012-01-09,2012-10-24,2,0
3,4,0,705108,2019-02-07,2019-02-07,1,0
4,5,0,705109,2019-03-13,2020-03-19,2,0


In [90]:
drug_era.shape

(5593841, 7)

In [91]:
drug_era.drug_concept_id.unique().size

3887

In [92]:
drug_era.drug_exposure_count.value_counts()

1    4740060
2     735565
3     103692
4      12952
5       1409
6        156
7          7
Name: drug_exposure_count, dtype: int64

In [93]:
drug_era.person_id.value_counts()

4288    724
5542    724
72      721
8688    720
8589    719
       ... 
4793      7
3929      3
5931      3
400       2
4901      1
Name: person_id, Length: 9134, dtype: int64

In [94]:
#drug_era.drug_concept_id.value_counts()
drug_era_count_data = drug_era.pivot_table(values='drug_exposure_count', index='person_id', columns='drug_concept_id', fill_value=0, aggfunc='mean')
drug_era_count_data.head()

drug_concept_id,501343,509104,523204,528325,528327,529411,529686,535714,535811,537649,543287,551749,551977,559532,701322,702867,703244,703248,703547,704599,704600,704601,704943,704944,704945,705103,705108,705109,705110,705157,705178,705219,705944,706237,706239,708298,710062,710650,711584,711588,711589,711644,711714,712615,712620,712621,712642,712643,713192,713823,713830,713831,713852,713853,713854,714684,714819,714850,714851,715233,715235,715236,715259,715294,715300,715734,715939,715940,715962,715997,716968,717136,717607,718122,718126,718127,718128,718129,718583,718669,718670,718757,718758,719174,719311,719380,719416,720730,720731,720810,721724,721760,722031,722067,722069,722070,722071,722424,722453,723013,723020,723042,725131,725178,725179,725180,730548,730729,732893,733008,733301,733562,734275,734303,734304,734354,734356,734360,734392,734394,734396,734400,734711,735533,735843,735850,735851,735882,735951,735979,736015,736020,736021,736053,736054,736055,736134,738156,738818,739138,739207,739209,739323,740275,740281,740394,740910,741572,741575,742185,742186,742188,742189,742190,742267,742268,742303,742304,743670,743717,743718,743719,743720,743721,743752,743753,744740,744798,744800,749910,749932,750982,751247,751249,751250,751251,751347,751362,751412,751889,751890,752061,752092,752095,752205,752209,753626,755695,755700,755739,755741,756018,757356,757358,757640,757688,757690,757691,757712,759401,759740,766529,766530,766814,766819,766820,766821,766842,766850,766923,766925,766927,777221,778268,778296,778299,778357,778474,778478,778479,778504,778505,778711,778736,778765,778770,780370,780383,780442,780444,781039,781062,781063,781126,781127,781130,781152,781182,781184,782211,785649,785788,785815,785821,785842,789580,789581,789612,790253,790255,...,42707501,42707566,42707635,42707649,42707716,42707819,42707912,42707985,42708082,42708086,42708092,42708142,42708144,42708166,42708174,42708258,42708297,42708308,42708359,42708386,42708389,42708415,42708472,42708510,42708512,42708518,42708523,42708584,42708623,42708624,42708645,42708660,42708733,42708737,42708743,42708824,42708867,42709103,42709138,42709140,42709180,42709282,42799049,42799119,42799215,42799331,42799410,42799649,42799711,42799739,42799785,42799833,42799842,42799903,42800046,42800050,42800072,42800173,42800184,42800186,42800295,42800296,42800300,42800355,42800473,42800591,42800620,42800634,42800967,42800984,42800988,42800992,42801011,42801149,42801287,42873430,42873440,42873444,42873452,42873476,42873483,42873503,42873596,42873600,42873604,42873622,42873636,42873638,42873673,42873851,42874036,42874135,42874220,42874224,42874248,42898499,42898503,42899056,42899058,42899398,42899453,42899470,42900250,42900252,42900359,42900505,42900507,42900511,42900586,42901645,42901649,42901669,42901674,42901705,42901722,42901844,42901846,42901848,42901850,42901885,42901964,42902110,42902114,42902168,42902177,42902216,42902231,42902236,42902239,42902244,42902282,42902295,42903460,42904205,43011783,43011850,43011897,43011910,43011929,43011932,43011962,43011966,43011980,43012036,43012292,43012432,43012434,43012446,43012520,43012671,43012750,43012797,43013021,43013024,43013026,43013030,43013071,43013182,43013581,43013597,43013637,43013656,43013896,43013911,43013924,43014087,43014091,43014144,43014237,43014243,43014251,43525648,43525679,43525681,43525695,43525720,43525787,43525972,43526209,43526401,43526430,43526467,43526471,43526482,43526510,43526514,43526602,43526851,43527008,43527216,43531768,43531861,43532944,43532949,43559947,43559969,43559982,43559983,43559996,43560301,43560385,43560387,43560392,43560429,43560431,43560433,43560442,43560451,43560590,43560658,43560714,44506611,44506612,44506613,44506703,44506752,44507529,44507558,44507566,44507580,44507582,44507597,44507676,44507848,44785094,44785907,44814542,44818461,44818489,45774613,45774639,45774751,45774861,45775116,45775372,45775636,45775965,45776670,45776944,45892075,45892326,45892419,45892531,45892628,45892833,45892847,46221284,4622143

In [95]:
drug_era_count_data = drug_era_count_data.add_prefix('drug_exposure_count_')
drug_era_count_data.reset_index(level=0, inplace=True)
drug_era_count_data.columns


Index(['person_id', 'drug_exposure_count_501343', 'drug_exposure_count_509104',
       'drug_exposure_count_523204', 'drug_exposure_count_528325',
       'drug_exposure_count_528327', 'drug_exposure_count_529411',
       'drug_exposure_count_529686', 'drug_exposure_count_535714',
       'drug_exposure_count_535811',
       ...
       'drug_exposure_count_45892326', 'drug_exposure_count_45892419',
       'drug_exposure_count_45892531', 'drug_exposure_count_45892628',
       'drug_exposure_count_45892833', 'drug_exposure_count_45892847',
       'drug_exposure_count_46221284', 'drug_exposure_count_46221433',
       'drug_exposure_count_46234437', 'drug_exposure_count_46274523'],
      dtype='object', name='drug_concept_id', length=3888)

In [96]:
drug_era.person_id.value_counts()

4288    724
5542    724
72      721
8688    720
8589    719
       ... 
4793      7
3929      3
5931      3
400       2
4901      1
Name: person_id, Length: 9134, dtype: int64

In [97]:
drug_era.shape

(5593841, 7)

In [98]:
drug_exposure = pd.read_csv("drug_exposure.csv", sep=',')
drug_exposure.head()


,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,1,4281,40169138,2016-12-04,2016-12-04,2016-12-06,2016-12-06,NaN,38000177,NaN,0.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40169138,Oral,NaN
1,2,4524,736021,2011-11-12,2011-11-12,2011-11-13,2011-11-13,NaN,38000177,NaN,1.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736021,Oral,mg
2,3,4076,1309800,2018-07-11,2018-07-11,2018-07-16,2018-07-16,NaN,38000177,NaN,0.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1309800,Oral,mg
3,4,421,36224549,2019-01-15,2019-01-15,2019-01-19,2019-01-19,NaN,43542358,Physician Stop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36224549,NaN,NaN
4,5,5010,1140650,2009-12-29,2009-12-29,2010-01-02,2010-01-02,NaN,38000177,NaN,3.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1140650,Oral,NaN


In [99]:
drug_exp_dose_unit_onehot_columns = ["dose_unit_source_value"]
onehot_columns_prefix = ["dose_unit_source_value"]

drug_exp_dose_unit_data = pd.get_dummies(drug_exposure,prefix=onehot_columns_prefix, columns=drug_exp_dose_unit_onehot_columns)
drug_exp_dose_unit_data.shape




(6582104, 103)

In [100]:
drug_exp_dose_unit_data.shape

(6582104, 103)

In [101]:
#drug_exp_dose_unit_onehot_columns = ["drug_concept_id"]
#onehot_columns_prefix = ["drug_concept_id_"]

#drug_exp_dose_unit_drug_data = pd.get_dummies(drug_exp_dose_unit_data,prefix=onehot_columns_prefix, columns=drug_exp_dose_unit_onehot_columns)
#drug_exp_dose_unit_drug_data.shape




In [102]:
del drug_exp_dose_unit_data["drug_exposure_id"]
del drug_exp_dose_unit_data["drug_exposure_start_date"]
del drug_exp_dose_unit_data["drug_exposure_start_datetime"]
del drug_exp_dose_unit_data["drug_exposure_end_date"]
del drug_exp_dose_unit_data["drug_exposure_end_datetime"]
del drug_exp_dose_unit_data["verbatim_end_date"]
del drug_exp_dose_unit_data["days_supply"]
del drug_exp_dose_unit_data["sig"]
del drug_exp_dose_unit_data["route_concept_id"]
del drug_exp_dose_unit_data["lot_number"]
del drug_exp_dose_unit_data["provider_id"]
del drug_exp_dose_unit_data["visit_occurrence_id"]
del drug_exp_dose_unit_data["visit_detail_id"]
del drug_exp_dose_unit_data["drug_source_value"]
del drug_exp_dose_unit_data["drug_source_concept_id"]
del drug_exp_dose_unit_data["route_source_value"]
del drug_exp_dose_unit_data["stop_reason"]
del drug_exp_dose_unit_data["drug_concept_id"]
del drug_exp_dose_unit_data["drug_type_concept_id"]
del drug_exp_dose_unit_data["refills"]
del drug_exp_dose_unit_data["quantity"]







In [103]:
drug_exp_dose_unit_data.dose_unit_source_value_Swab.value_counts()

0    6581912
1        192
Name: dose_unit_source_value_Swab, dtype: int64

In [104]:
drug_exp_dose_unit_data_final = drug_exp_dose_unit_data.groupby('person_id').agg(sum)
#processed_condition_final['person_id'] = processed_condition_final.index
#drug_exp_dose_unit_data.reset_index(level=0, inplace=True)


In [105]:
drug_exp_dose_unit_data_final.head()

,dose_unit_source_value_AUC,dose_unit_source_value_Act,dose_unit_source_value_Aerosol,dose_unit_source_value_Application,dose_unit_source_value_Applicatorful,dose_unit_source_value_Bottle,dose_unit_source_value_Box,dose_unit_source_value_Can,dose_unit_source_value_Capsule 12 Hour Sustained Release,dose_unit_source_value_Capsule 24 Hour Sustained Release,dose_unit_source_value_Cartridge,dose_unit_source_value_Dose,dose_unit_source_value_Drop,dose_unit_source_value_Each,dose_unit_source_value_Enema,dose_unit_source_value_Film,dose_unit_source_value_Implant,dose_unit_source_value_Inch,dose_unit_source_value_Inhalant,dose_unit_source_value_Inhalation,dose_unit_source_value_Injectable,dose_unit_source_value_Int'l Units,dose_unit_source_value_Intra Uterine Device,dose_unit_source_value_Kit,dose_unit_source_value_L,dose_unit_source_value_L/day,dose_unit_source_value_Lozenge,dose_unit_source_value_MU,dose_unit_source_value_Million Units,dose_unit_source_value_Nebule,dose_unit_source_value_Pack,dose_unit_source_value_Package,dose_unit_source_value_Packet,dose_unit_source_value_Pad,dose_unit_source_value_Patch,dose_unit_source_value_Piece,dose_unit_source_value_Powder for Reconstitution,dose_unit_source_value_Puff,dose_unit_source_value_Ring,dose_unit_source_value_Scoop,dose_unit_source_value_Spray,dose_unit_source_value_Strip,dose_unit_source_value_Sublingual Tablet,dose_unit_source_value_Suppository,dose_unit_source_value_Swab,dose_unit_source_value_Syringe,dose_unit_source_value_Tablet 12 Hour Sustained Release,dose_unit_source_value_Tablet 24 Hour Sustained Release,dose_unit_source_value_Tube,dose_unit_source_value_Units,dose_unit_source_value_Vial,dose_unit_source_value_ampule,dose_unit_source_value_caplet,dose_unit_source_value_capsule,dose_unit_source_value_each,dose_unit_source_value_g,dose_unit_source_value_g/100 mL,dose_unit_source_value_g/kg,dose_unit_source_value_inch,dose_unit_source_value_mEq,dose_unit_source_value_mL,dose_unit_source_value_mL/kg,dose_unit_source_value_mcg,dose_unit_source_value_mcg/hr,dose_unit_source_value_mcg/kg/min,dose_unit_source_value_mcg/min,dose_unit_source_value_mg,dose_unit_source_value_mg of ELEMENTAL calcium,dose_unit_source_value_mg of ELEMENTAL iron,dose_unit_source_value_mg/hr,dose_unit_source_value_mg/kg,dose_unit_source_value_mg/kg/day,dose_unit_source_value_mg/m2,dose_unit_source_value_mgPE,dose_unit_source_value_ng,dose_unit_source_value_pump,dose_unit_source_value_suppository,dose_unit_source_value_tablet,dose_unit_source_value_units,dose_unit_source_value_units/hr,dose_unit_source_value_units/kg
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,3.0,1.0,11.0,0.0,0.0,0.0,4.0,7.0,0.0,6.0,0.0,0.0,0.0,215.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0,0.0,0.0,0.0
1,0.0,0.0,0.0,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,8.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,1.0,8.0,0.0,0.0,0.0,1.0,3.0,0.0,9.0,0.0,0.0,0.0,242.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,29.0,1.0,0.0,0.0
2,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,0.0,2.0,9.0,0.0,8.0,0.0,0.0,0.0,235.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,29.0,2.0,0.0,0.0
3,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0,0.0,17.0,1.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,4.0,0.0,10.0,0.0,0.0,1.0,3.0,5.0,0.0,10.0,0.0,0.0,0.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20.0,1.0,0.0,0.0

In [106]:
drug_exp_dose_unit_data_final.shape

(9134, 81)

In [107]:
#drug_exp_dose_unit_data_final.loc[drug_exp_dose_unit_data_final > 0] = 1
columns = drug_exp_dose_unit_data_final.columns
for col in columns:
    if col != "person_id":
        drug_exp_dose_unit_data_final[col] = drug_exp_dose_unit_data_final[col].apply(lambda x: 1 if x >0 else 0)


In [108]:
drug_exp_dose_unit_data_final.head()

,dose_unit_source_value_AUC,dose_unit_source_value_Act,dose_unit_source_value_Aerosol,dose_unit_source_value_Application,dose_unit_source_value_Applicatorful,dose_unit_source_value_Bottle,dose_unit_source_value_Box,dose_unit_source_value_Can,dose_unit_source_value_Capsule 12 Hour Sustained Release,dose_unit_source_value_Capsule 24 Hour Sustained Release,dose_unit_source_value_Cartridge,dose_unit_source_value_Dose,dose_unit_source_value_Drop,dose_unit_source_value_Each,dose_unit_source_value_Enema,dose_unit_source_value_Film,dose_unit_source_value_Implant,dose_unit_source_value_Inch,dose_unit_source_value_Inhalant,dose_unit_source_value_Inhalation,dose_unit_source_value_Injectable,dose_unit_source_value_Int'l Units,dose_unit_source_value_Intra Uterine Device,dose_unit_source_value_Kit,dose_unit_source_value_L,dose_unit_source_value_L/day,dose_unit_source_value_Lozenge,dose_unit_source_value_MU,dose_unit_source_value_Million Units,dose_unit_source_value_Nebule,dose_unit_source_value_Pack,dose_unit_source_value_Package,dose_unit_source_value_Packet,dose_unit_source_value_Pad,dose_unit_source_value_Patch,dose_unit_source_value_Piece,dose_unit_source_value_Powder for Reconstitution,dose_unit_source_value_Puff,dose_unit_source_value_Ring,dose_unit_source_value_Scoop,dose_unit_source_value_Spray,dose_unit_source_value_Strip,dose_unit_source_value_Sublingual Tablet,dose_unit_source_value_Suppository,dose_unit_source_value_Swab,dose_unit_source_value_Syringe,dose_unit_source_value_Tablet 12 Hour Sustained Release,dose_unit_source_value_Tablet 24 Hour Sustained Release,dose_unit_source_value_Tube,dose_unit_source_value_Units,dose_unit_source_value_Vial,dose_unit_source_value_ampule,dose_unit_source_value_caplet,dose_unit_source_value_capsule,dose_unit_source_value_each,dose_unit_source_value_g,dose_unit_source_value_g/100 mL,dose_unit_source_value_g/kg,dose_unit_source_value_inch,dose_unit_source_value_mEq,dose_unit_source_value_mL,dose_unit_source_value_mL/kg,dose_unit_source_value_mcg,dose_unit_source_value_mcg/hr,dose_unit_source_value_mcg/kg/min,dose_unit_source_value_mcg/min,dose_unit_source_value_mg,dose_unit_source_value_mg of ELEMENTAL calcium,dose_unit_source_value_mg of ELEMENTAL iron,dose_unit_source_value_mg/hr,dose_unit_source_value_mg/kg,dose_unit_source_value_mg/kg/day,dose_unit_source_value_mg/m2,dose_unit_source_value_mgPE,dose_unit_source_value_ng,dose_unit_source_value_pump,dose_unit_source_value_suppository,dose_unit_source_value_tablet,dose_unit_source_value_units,dose_unit_source_value_units/hr,dose_unit_source_value_units/kg
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0
1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [109]:
#drug_exp_dose_unit_data.head()
drug_exp_dose_unit_data_final.reset_index(level=0, inplace=True)
#columns = drug_exp_dose_unit_data_final.columns
#for col in columns:
#    print(col)
drug_exp_dose_unit_data_final.head()

,person_id,dose_unit_source_value_AUC,dose_unit_source_value_Act,dose_unit_source_value_Aerosol,dose_unit_source_value_Application,dose_unit_source_value_Applicatorful,dose_unit_source_value_Bottle,dose_unit_source_value_Box,dose_unit_source_value_Can,dose_unit_source_value_Capsule 12 Hour Sustained Release,dose_unit_source_value_Capsule 24 Hour Sustained Release,dose_unit_source_value_Cartridge,dose_unit_source_value_Dose,dose_unit_source_value_Drop,dose_unit_source_value_Each,dose_unit_source_value_Enema,dose_unit_source_value_Film,dose_unit_source_value_Implant,dose_unit_source_value_Inch,dose_unit_source_value_Inhalant,dose_unit_source_value_Inhalation,dose_unit_source_value_Injectable,dose_unit_source_value_Int'l Units,dose_unit_source_value_Intra Uterine Device,dose_unit_source_value_Kit,dose_unit_source_value_L,dose_unit_source_value_L/day,dose_unit_source_value_Lozenge,dose_unit_source_value_MU,dose_unit_source_value_Million Units,dose_unit_source_value_Nebule,dose_unit_source_value_Pack,dose_unit_source_value_Package,dose_unit_source_value_Packet,dose_unit_source_value_Pad,dose_unit_source_value_Patch,dose_unit_source_value_Piece,dose_unit_source_value_Powder for Reconstitution,dose_unit_source_value_Puff,dose_unit_source_value_Ring,dose_unit_source_value_Scoop,dose_unit_source_value_Spray,dose_unit_source_value_Strip,dose_unit_source_value_Sublingual Tablet,dose_unit_source_value_Suppository,dose_unit_source_value_Swab,dose_unit_source_value_Syringe,dose_unit_source_value_Tablet 12 Hour Sustained Release,dose_unit_source_value_Tablet 24 Hour Sustained Release,dose_unit_source_value_Tube,dose_unit_source_value_Units,dose_unit_source_value_Vial,dose_unit_source_value_ampule,dose_unit_source_value_caplet,dose_unit_source_value_capsule,dose_unit_source_value_each,dose_unit_source_value_g,dose_unit_source_value_g/100 mL,dose_unit_source_value_g/kg,dose_unit_source_value_inch,dose_unit_source_value_mEq,dose_unit_source_value_mL,dose_unit_source_value_mL/kg,dose_unit_source_value_mcg,dose_unit_source_value_mcg/hr,dose_unit_source_value_mcg/kg/min,dose_unit_source_value_mcg/min,dose_unit_source_value_mg,dose_unit_source_value_mg of ELEMENTAL calcium,dose_unit_source_value_mg of ELEMENTAL iron,dose_unit_source_value_mg/hr,dose_unit_source_value_mg/kg,dose_unit_source_value_mg/kg/day,dose_unit_source_value_mg/m2,dose_unit_source_value_mgPE,dose_unit_source_value_ng,dose_unit_source_value_pump,dose_unit_source_value_suppository,dose_unit_source_value_tablet,dose_unit_source_value_units,dose_unit_source_value_units/hr,dose_unit_source_value_units/kg
0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0
1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0
2,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0
3,3,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [110]:
drug_exp_dose_unit_data_final.shape

(9134, 82)

In [111]:
drug_exposure_quantity_data = drug_exposure.pivot_table(values='quantity', index='person_id', columns='drug_concept_id', fill_value=0, aggfunc='mean')
drug_exposure_quantity_data.head()

drug_concept_id,523204,702867,703248,703547,704601,704943,704944,704945,705108,705109,705110,705944,711584,711588,711589,711644,712620,712621,712642,712643,713830,713831,713852,713853,713854,714819,714850,714851,715235,715236,715294,715300,715939,715940,715962,718127,718128,718129,718583,718669,718670,718757,718758,719380,719416,720730,720731,721760,722067,722069,722070,722071,722453,723020,723042,725131,725178,725179,725180,733562,734303,734304,734354,734356,734360,734392,734394,734396,734400,735533,735851,735882,735979,736015,736020,736021,736053,736054,736055,739138,739207,739209,740281,740394,740910,741572,741575,742186,742188,742189,742268,742303,742304,743670,743717,743718,743719,743720,743721,743752,743753,744740,744800,749932,750982,751247,751249,751250,751251,751362,752061,752092,752095,752205,755695,755700,755739,757356,757358,757690,757691,757712,766819,766820,766821,766842,766850,766923,766927,777221,778296,778299,778357,778478,778479,778504,778505,778711,778736,778765,780383,780444,781039,781062,781063,781126,781127,781130,781184,785788,785815,785821,785842,789580,789581,790255,791967,792000,792295,792929,797399,797415,797439,797617,797632,798874,798875,798876,798877,798913,798914,798917,800881,800892,836718,836719,836720,902059,902196,902429,902435,902489,902740,902741,902806,902971,903052,903222,903711,903753,904435,904457,904490,904492,904546,904549,905236,905266,905294,906780,906891,906917,906921,906933,906973,907016,907020,907071,908726,908944,908948,908982,909647,911065,911101,911130,912267,916006,916008,916499,916774,917207,917310,917337,917338,917339,917552,918363,918543,918634,918906,920332,920334,920949,922570,922802,922806,923115,923645,923647,923720,923721,923876,924172,924566,924939,929519,929889,929924,932745,...,40226963,40226992,40226995,40226998,40227060,40227085,40227101,40227543,40227546,40228076,40228087,40228154,40228161,40228203,40228212,40228214,40228230,40228841,40229058,40229445,40230465,40230469,40231144,40231370,40231374,40231516,40231630,40231761,40231804,40231806,40231810,40231812,40231814,40231816,40231818,40231822,40231855,40231916,40231925,40231932,40232217,40232259,40232435,40232439,40232444,40232448,40232612,40232618,40232649,40232700,40232707,40232717,40232730,40232745,40232754,40232756,40233220,40233894,40233955,40233964,40233977,40233993,40234030,40234084,40234301,40234552,40234705,40234727,40234838,40234842,40234846,40235212,40235257,40235260,40235269,40235274,40235279,40235281,40235286,40235896,40236419,40236446,40236479,40236481,40236648,40236691,40236824,40236839,40237418,40237432,40237517,40237527,40237529,40237543,40238005,40238008,40238021,40238115,40238221,40238240,40238246,40238254,40238415,40238625,40238670,40238932,40239011,40239060,40239109,40239218,40239551,40240035,40240039,40240187,40240377,40240688,40241038,40241046,40241178,40241183,40241188,40241331,40241333,40241488,40241504,40241536,40241958,40241982,40242221,40242522,40242539,40243202,40243340,40243447,40243494,40243835,40243855,40243859,40243864,40243866,40243872,40243887,40243919,40244256,40244338,40244444,40244448,40244464,42707259,42707354,42707635,42707649,42707985,42708297,42708389,42708415,42708510,42708512,42708623,42708624,42708660,42708743,42709103,42709138,42709180,42709282,42799331,42799410,42799785,42799833,42799903,42800072,42800173,42800184,42800186,42800300,42800967,42800984,42801011,42801149,42873430,42873440,42873444,42873596,42873600,42873622,42873673,42873851,42874224,42874248,42898503,42900252,42900507,42900511,42900586,42901649,42901669,42901844,42901846,42901964,42902168,42902177,42902216,42902231,42902236,42902244,42902282,43011897,43011910,43011929,43012036,43012432,43012434,43012671,43012797,43013021,43013024,43013026,43013030,43013637,43013911,43014087,43014091,43014144,43014237,43525681,43525695,43525972,43526209,43526401,43526430,43526467,43526471,43526514,43526851,43531861,43532944,43532949,43559947,43559969,43559982,43559996,43560301,43560387,43560433,43560451,44506703,4450756

In [112]:
drug_exposure_quantity_data = drug_exposure_quantity_data.add_prefix('drug_quantity_')
drug_exposure_quantity_data.reset_index(level=0, inplace=True)
drug_exposure_quantity_data.columns

Index(['person_id', 'drug_quantity_523204', 'drug_quantity_702867',
       'drug_quantity_703248', 'drug_quantity_703547', 'drug_quantity_704601',
       'drug_quantity_704943', 'drug_quantity_704944', 'drug_quantity_704945',
       'drug_quantity_705108',
       ...
       'drug_quantity_43559982', 'drug_quantity_43559996',
       'drug_quantity_43560301', 'drug_quantity_43560387',
       'drug_quantity_43560433', 'drug_quantity_43560451',
       'drug_quantity_44506703', 'drug_quantity_44507566',
       'drug_quantity_44507582', 'drug_quantity_44507848'],
      dtype='object', name='drug_concept_id', length=2181)

In [113]:
drug_exposure_quantity_data.shape

(9134, 2181)

In [114]:
drug_exposure_refills_data = drug_exposure.pivot_table(values='refills', index='person_id', columns='drug_concept_id', fill_value=0, aggfunc='mean')
drug_exposure_refills_data.head()

drug_concept_id,523204,529686,703248,703547,704601,704943,704944,704945,705103,705108,705109,705110,711584,711588,711589,711644,712620,712621,712642,712643,713830,713831,713852,713853,713854,715235,715236,715259,715294,715300,715940,715962,718127,718128,718129,718583,718669,718670,718757,718758,720730,720731,721760,722067,722069,722070,722071,722453,723020,723042,725131,725178,725179,725180,733562,734303,734304,734354,734356,734360,734392,734394,734396,734400,735533,735851,735882,735979,736015,736020,736021,736053,736054,736055,739138,739207,739209,740281,740394,741572,741575,742186,742188,742189,742190,742268,742303,742304,743717,743718,743719,743720,743721,743752,744740,744800,749932,751247,751249,751250,751251,751362,751890,752061,752092,752095,752205,755700,755739,755741,757356,757358,757690,757691,757712,766819,766820,766821,766842,766850,766923,766927,777221,778296,778299,778357,778478,778479,778504,778505,778711,778736,778765,780383,780444,781062,781063,781126,781184,785788,785815,785821,785842,789580,789581,790255,791967,792000,792295,792929,797399,797415,797439,797617,797632,798875,798876,798877,798913,798914,800881,800892,836718,836719,836720,902059,902429,902435,902489,902741,902806,902971,903052,903222,903679,903711,903753,904435,904453,904457,904490,904492,904546,904547,904549,905236,905266,905294,906780,906891,906917,906921,906933,906973,907016,907020,907071,908726,908944,908948,908951,908982,909647,911065,911101,911130,912267,916006,916008,916462,916499,916774,917207,917310,917337,917338,917339,917552,918363,918543,918634,918906,919346,920332,920334,920949,922570,922802,922806,923115,923645,923647,923720,923721,924172,929153,929519,929889,929924,932745,932748,932749,932750,933724,938268,938330,938435,939300,939506,939509,...,40223901,40223932,40223992,40224108,40224320,40224399,40224433,40224479,40224655,40224789,40224812,40224822,40224835,40224875,40224881,40224897,40224903,40224944,40225082,40225159,40225691,40225695,40225989,40225993,40226429,40226581,40226963,40226992,40226995,40226998,40227060,40227085,40227101,40227428,40227525,40227543,40227546,40227569,40228087,40228154,40228161,40228203,40228212,40228214,40228230,40228841,40229058,40229445,40230465,40230469,40231144,40231370,40231374,40231516,40231630,40231761,40231822,40231925,40232217,40232259,40232435,40232439,40232444,40232448,40232612,40232649,40232700,40232756,40233220,40233894,40233955,40233964,40233977,40233993,40234019,40234084,40234301,40234552,40234705,40234727,40234842,40234846,40235286,40235896,40236481,40236648,40236691,40236824,40236839,40237312,40237432,40237517,40237527,40237529,40237543,40238008,40238021,40238115,40238221,40238240,40238246,40238254,40238415,40238625,40238932,40239011,40239060,40239109,40239218,40239551,40240035,40240039,40240187,40240377,40240666,40240688,40241038,40241178,40241183,40241188,40241333,40241488,40241536,40241982,40242522,40243202,40243206,40243340,40243447,40243494,40243835,40243855,40243859,40243864,40243866,40243872,40243887,40243919,40244256,40244338,40244444,40244448,40244464,42707259,42707354,42707635,42707649,42707985,42708166,42708297,42708389,42708415,42708510,42708512,42708523,42708623,42708624,42708660,42708743,42708867,42709103,42709138,42709180,42709282,42799049,42799331,42799410,42799785,42799833,42799903,42800072,42800173,42800184,42800186,42800300,42800620,42800967,42800984,42801011,42801149,42873430,42873440,42873444,42873622,42873673,42874224,42874248,42898503,42900252,42900507,42900511,42900586,42901649,42901705,42901844,42901848,42901964,42902168,42902177,42902216,42902231,42902236,42902244,42902282,43011897,43011910,43011929,43012036,43012432,43012434,43012671,43012797,43013021,43013024,43013026,43013030,43013637,43013911,43013924,43014087,43014091,43014144,43014237,43525681,43525695,43525972,43526209,43526401,43526430,43526467,43526471,43526514,43526851,43527008,43531768,43531861,43532944,43532949,43559947,43559996,43560301,43560387,43560451,44506611,44506613,44507566,44507582,4450759

In [115]:
drug_exposure_refills_data.shape

(9133, 2096)

In [116]:
drug_exposure_refills_data = drug_exposure_refills_data.add_prefix('drug_refills_')
drug_exposure_refills_data.reset_index(level=0, inplace=True)
drug_exposure_refills_data.columns

Index(['person_id', 'drug_refills_523204', 'drug_refills_529686',
       'drug_refills_703248', 'drug_refills_703547', 'drug_refills_704601',
       'drug_refills_704943', 'drug_refills_704944', 'drug_refills_704945',
       'drug_refills_705103',
       ...
       'drug_refills_43560301', 'drug_refills_43560387',
       'drug_refills_43560451', 'drug_refills_44506611',
       'drug_refills_44506613', 'drug_refills_44507566',
       'drug_refills_44507582', 'drug_refills_44507597',
       'drug_refills_44507848', 'drug_refills_45892075'],
      dtype='object', name='drug_concept_id', length=2097)

In [117]:
drug_exposure.refills.value_counts()

0.0     1593981
3.0     1060145
1.0      839363
2.0      541644
5.0      345229
11.0     243821
6.0      230034
4.0      144853
12.0      52640
10.0      14117
8.0        9918
13.0       5629
9.0        5010
99.0       2732
7.0        1893
60.0        199
20.0         23
15.0         14
Name: refills, dtype: int64

In [118]:
drug_exposure.dose_unit_source_value.value_counts()

mg                                   1904190
tablet                                200387
Application                           167354
mcg                                    87809
Drop                                   82206
                                      ...   
AUC                                      220
Aerosol                                  213
Capsule 12 Hour Sustained Release        213
units/kg                                 211
Swab                                     192
Name: dose_unit_source_value, Length: 81, dtype: int64

In [119]:
drug_exposure.shape

(6582104, 23)

In [120]:
drug_exposure.drug_concept_id.value_counts()



967823      10292
1000560      7742
1717707      7648
1518254      7582
40167259     7247
            ...  
1350506       187
40184712      186
40161247      185
19080181      183
950497        182
Name: drug_concept_id, Length: 3887, dtype: int64

In [121]:
drug_exposure.person_id.value_counts()

6565    864
6798    856
8612    856
2616    856
7476    851
       ... 
4793      7
5931      3
3929      3
400       2
4901      1
Name: person_id, Length: 9134, dtype: int64

In [122]:
drug_exposure.stop_reason.value_counts()

Physician Stop    177213
Maintenance       113533
Hard Stop          74669
Soft Stop          10217
Acute               2123
Name: stop_reason, dtype: int64

In [123]:
drug_quantity_refill =  pd.merge(drug_exposure_quantity_data,
                        drug_exposure_refills_data,#[['person_id', 'status']],
                        on='person_id',
                        how = 'left')


drug_quantity_refill.shape

(9134, 4277)

In [124]:
drug_quantity_refill_dose =  pd.merge(drug_quantity_refill,
                        drug_exp_dose_unit_data_final,#[['person_id', 'status']],
                        on='person_id',
                        how = 'left')


drug_quantity_refill_dose.shape

(9134, 4358)

In [125]:
device_exposure = pd.read_csv("device_exposure.csv", sep=',')
device_exposure.head()


,device_exposure_id,person_id,device_concept_id,device_exposure_start_date,device_exposure_start_datetime,device_exposure_end_date,device_exposure_end_datetime,device_type_concept_id,unique_device_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,device_source_value,device_source_concept_id
0,1,4225,45768197,2015-05-13,2015-05-13 23:24:00,2015-05-18,2015-05-18 23:24:00,44818707,NaN,1,NaN,NaN,NaN,NaN,45768197
1,2,124,45768197,2017-04-10,2017-04-10 23:24:00,2017-04-12,2017-04-12 23:24:00,44818707,NaN,1,NaN,NaN,NaN,NaN,45768197
2,3,7959,45768197,2015-11-19,2015-11-19 23:24:00,2015-11-20,2015-11-20 23:24:00,44818707,NaN,1,NaN,NaN,NaN,NaN,45768197
3,4,2096,45768197,2013-09-07,2013-09-07 23:24:00,2013-09-12,2013-09-12 23:24:00,44818707,NaN,1,NaN,NaN,NaN,NaN,45768197
4,5,8007,45768197,2011-04-26,2011-04-26 23:24:00,2011-04-29,2011-04-29 23:24:00,44818707,NaN,1,NaN,NaN,NaN,NaN,45768197


In [126]:
device_exposure.person_id.unique().size

833

In [127]:
device_exposure.shape

(884, 15)

In [128]:
device_exposure.device_type_concept_id.unique().size

1

In [129]:
device_exposure_subset = device_exposure[["person_id","device_concept_id"]]


In [130]:
#device_exposure.person_id.value_counts()
#device_exposure.info()
device_exposure.person_id.size

884

In [131]:
#location = pd.read_csv("location.csv", sep=',')
#location.head()



In [132]:
measurement = pd.read_csv("measurement.csv", sep=',')
measurement.head()



,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_time,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value
0,1,3569,3007220,2015-05-16,2015-05-16 09:58:00,2019-05-16,44818702,4172703.0,23.00,NaN,8645.0,43.00,274.00,NaN,NaN,NaN,NaN,3007220,U/L,23.00
1,2,2551,3015242,2015-04-05,2015-04-05 09:58:00,2019-08-30,44818702,4172703.0,586.00,NaN,8842.0,10.00,180.00,NaN,NaN,NaN,NaN,3015242,ng/mL,586.00
2,3,6649,3006923,2010-06-15,2010-06-15 09:58:00,2018-11-29,44818702,4172703.0,24.00,NaN,8645.0,10.00,64.00,NaN,NaN,NaN,NaN,3006923,U/L,24.00
3,4,640,3016723,2015-02-27,2015-02-27 09:58:00,2016-11-11,44818702,4172703.0,0.67,NaN,8840.0,0.51,1.18,NaN,NaN,NaN,NaN,3016723,mg/dL,0.67
4,5,8739,3006923,2012-11-27,2012-11-27 09:58:00,2019-02-28,44818702,4172703.0,46.00,NaN,8645.0,10.00,64.00,NaN,NaN,NaN,NaN,3006923,U/L,46.00


In [133]:
measurement.shape

(6376178, 20)

In [134]:
#measurement_subset.measurement_type_concept_id.value_counts()

In [135]:
#measurement.person_id.value_counts()

measurement_subset = measurement[["person_id","measurement_concept_id", "measurement_type_concept_id"]]


In [136]:
measurement_onehot_columns = ["measurement_type_concept_id"]
onehot_columns_prefix = ["measurement_type_concept_id"]

measurement_data_typeid = pd.get_dummies(measurement_subset,prefix=onehot_columns_prefix, columns=measurement_onehot_columns)
measurement_data_typeid.shape

#measurement_subset

(6376178, 4)

In [137]:
measurement_data_typeid.head()


,person_id,measurement_concept_id,measurement_type_concept_id_3028553,measurement_type_concept_id_44818702
0,3569,3007220,0,1
1,2551,3015242,0,1
2,6649,3006923,0,1
3,640,3016723,0,1
4,8739,3006923,0,1


In [138]:
measurement_onehot_columns = ["measurement_concept_id"]
onehot_columns_prefix = ["measurement_concept_id"]

measurement_data = pd.get_dummies(measurement_data_typeid,prefix=onehot_columns_prefix, columns=measurement_onehot_columns)
measurement_data.shape


(6376178, 70)

In [139]:
measurement_data.head()

,person_id,measurement_type_concept_id_3028553,measurement_type_concept_id_44818702,measurement_concept_id_3000905,measurement_concept_id_3000963,measurement_concept_id_3003396,measurement_concept_id_3003694,measurement_concept_id_3004249,measurement_concept_id_3004327,measurement_concept_id_3004501,measurement_concept_id_3005491,measurement_concept_id_3006262,measurement_concept_id_3006923,measurement_concept_id_3007220,measurement_concept_id_3008037,measurement_concept_id_3008152,measurement_concept_id_3010156,measurement_concept_id_3011960,measurement_concept_id_3012158,measurement_concept_id_3012544,measurement_concept_id_3012888,measurement_concept_id_3013650,measurement_concept_id_3013682,measurement_concept_id_3013707,measurement_concept_id_3013721,measurement_concept_id_3014576,measurement_concept_id_3015242,measurement_concept_id_3016407,measurement_concept_id_3016502,measurement_concept_id_3016723,measurement_concept_id_3018405,measurement_concept_id_3018677,measurement_concept_id_3019550,measurement_concept_id_3019897,measurement_concept_id_3019977,measurement_concept_id_3020716,measurement_concept_id_3020891,measurement_concept_id_3021337,measurement_concept_id_3022192,measurement_concept_id_3022250,measurement_concept_id_3023091,measurement_concept_id_3023103,measurement_concept_id_3023314,measurement_concept_id_3023548,measurement_concept_id_3024128,measurement_concept_id_3024171,measurement_concept_id_3024561,measurement_concept_id_3024929,measurement_concept_id_3025023,measurement_concept_id_3025315,measurement_concept_id_3025634,measurement_concept_id_3027018,measurement_concept_id_3027114,measurement_concept_id_3027801,measurement_concept_id_3027946,measurement_concept_id_3028167,measurement_concept_id_3033891,measurement_concept_id_3036277,measurement_concept_id_3038288,measurement_concept_id_3038297,measurement_concept_id_3041623,measurement_concept_id_3042194,measurement_concept_id_3042596,measurement_concept_id_3044254,measurement_concept_id_3044938,measurement_concept_id_3045716,measurement_concept_id_3046279,measurement_concept_id_4196147,measurement_concept_id_40765161,measurement_concept_id_42870366
0,3569,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2551,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6649,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,640,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8739,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [140]:
measurement_data_grouped = measurement_data.groupby('person_id').agg(sum)
#processed_condition_final['person_id'] = processed_condition_final.index

measurement_data_grouped.reset_index(level=0, inplace=True)


In [141]:
measurement_data_grouped.head()

,person_id,measurement_type_concept_id_3028553,measurement_type_concept_id_44818702,measurement_concept_id_3000905,measurement_concept_id_3000963,measurement_concept_id_3003396,measurement_concept_id_3003694,measurement_concept_id_3004249,measurement_concept_id_3004327,measurement_concept_id_3004501,measurement_concept_id_3005491,measurement_concept_id_3006262,measurement_concept_id_3006923,measurement_concept_id_3007220,measurement_concept_id_3008037,measurement_concept_id_3008152,measurement_concept_id_3010156,measurement_concept_id_3011960,measurement_concept_id_3012158,measurement_concept_id_3012544,measurement_concept_id_3012888,measurement_concept_id_3013650,measurement_concept_id_3013682,measurement_concept_id_3013707,measurement_concept_id_3013721,measurement_concept_id_3014576,measurement_concept_id_3015242,measurement_concept_id_3016407,measurement_concept_id_3016502,measurement_concept_id_3016723,measurement_concept_id_3018405,measurement_concept_id_3018677,measurement_concept_id_3019550,measurement_concept_id_3019897,measurement_concept_id_3019977,measurement_concept_id_3020716,measurement_concept_id_3020891,measurement_concept_id_3021337,measurement_concept_id_3022192,measurement_concept_id_3022250,measurement_concept_id_3023091,measurement_concept_id_3023103,measurement_concept_id_3023314,measurement_concept_id_3023548,measurement_concept_id_3024128,measurement_concept_id_3024171,measurement_concept_id_3024561,measurement_concept_id_3024929,measurement_concept_id_3025023,measurement_concept_id_3025315,measurement_concept_id_3025634,measurement_concept_id_3027018,measurement_concept_id_3027114,measurement_concept_id_3027801,measurement_concept_id_3027946,measurement_concept_id_3028167,measurement_concept_id_3033891,measurement_concept_id_3036277,measurement_concept_id_3038288,measurement_concept_id_3038297,measurement_concept_id_3041623,measurement_concept_id_3042194,measurement_concept_id_3042596,measurement_concept_id_3044254,measurement_concept_id_3044938,measurement_concept_id_3045716,measurement_concept_id_3046279,measurement_concept_id_4196147,measurement_concept_id_40765161,measurement_concept_id_42870366
0,0,322.0,446.0,66.0,9.0,4.0,0.0,82.0,19.0,15.0,0.0,0.0,27.0,8.0,0.0,0.0,18.0,16.0,0.0,3.0,73.0,51.0,6.0,5.0,12.0,1.0,29.0,8.0,0.0,30.0,1.0,3.0,1.0,11.0,0.0,0.0,7.0,3.0,12.0,13.0,2.0,0.0,3.0,3.0,0.0,6.0,2.0,18.0,0.0,120.0,0.0,10.0,10.0,17.0,0.0,9.0,1.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,6.0
1,1,310.0,414.0,57.0,7.0,4.0,0.0,63.0,10.0,18.0,0.0,0.0,22.0,12.0,2.0,1.0,16.0,13.0,0.0,1.0,91.0,47.0,4.0,2.0,12.0,0.0,18.0,8.0,0.0,27.0,4.0,3.0,3.0,11.0,3.0,1.0,4.0,2.0,9.0,21.0,0.0,1.0,3.0,1.0,3.0,2.0,2.0,24.0,0.0,103.0,0.0,12.0,5.0,17.0,0.0,9.0,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,5.0
2,2,329.0,401.0,47.0,7.0,3.0,0.0,64.0,13.0,15.0,1.0,1.0,18.0,15.0,1.0,2.0,17.0,20.0,0.0,1.0,93.0,40.0,4.0,3.0,11.0,0.0,16.0,5.0,0.0,26.0,2.0,6.0,0.0,8.0,1.0,0.0,7.0,3.0,15.0,13.0,2.0,1.0,3.0,4.0,1.0,7.0,1.0,21.0,0.0,114.0,0.0,11.0,5.0,21.0,4.0,10.0,1.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12.0
3,3,354.0,395.0,62.0,9.0,2.0,0.0,86.0,13.0,19.0,0.0,0.0,21.0,15.0,2.0,1.0,10.0,8.0,0.0,1.0,86.0,34.0,2.0,3.0,8.0,4.0,22.0,9.0,0.0,28.0,3.0,3.0,1.0,6.0,1.0,2.0,10.0,4.0,8.0,17.0,2.0,1.0,2.0,3.0,6.0,3.0,0.0,13.0,0.0,136.0,0.0,10.0,6.0,22.0,2.0,12.0,1.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,5.0
4,4,11.0,11.0,1.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [142]:
measurement_data_grouped.shape

(9133, 70)

In [143]:
columns = measurement_data_grouped.columns
for col in columns:
    if col != "person_id":
        measurement_data_grouped[col] = measurement_data_grouped[col].apply(lambda x: 1 if x >0 else 0)
#if answer == 'correct':

In [144]:
measurement_data_grouped.head()

,person_id,measurement_type_concept_id_3028553,measurement_type_concept_id_44818702,measurement_concept_id_3000905,measurement_concept_id_3000963,measurement_concept_id_3003396,measurement_concept_id_3003694,measurement_concept_id_3004249,measurement_concept_id_3004327,measurement_concept_id_3004501,measurement_concept_id_3005491,measurement_concept_id_3006262,measurement_concept_id_3006923,measurement_concept_id_3007220,measurement_concept_id_3008037,measurement_concept_id_3008152,measurement_concept_id_3010156,measurement_concept_id_3011960,measurement_concept_id_3012158,measurement_concept_id_3012544,measurement_concept_id_3012888,measurement_concept_id_3013650,measurement_concept_id_3013682,measurement_concept_id_3013707,measurement_concept_id_3013721,measurement_concept_id_3014576,measurement_concept_id_3015242,measurement_concept_id_3016407,measurement_concept_id_3016502,measurement_concept_id_3016723,measurement_concept_id_3018405,measurement_concept_id_3018677,measurement_concept_id_3019550,measurement_concept_id_3019897,measurement_concept_id_3019977,measurement_concept_id_3020716,measurement_concept_id_3020891,measurement_concept_id_3021337,measurement_concept_id_3022192,measurement_concept_id_3022250,measurement_concept_id_3023091,measurement_concept_id_3023103,measurement_concept_id_3023314,measurement_concept_id_3023548,measurement_concept_id_3024128,measurement_concept_id_3024171,measurement_concept_id_3024561,measurement_concept_id_3024929,measurement_concept_id_3025023,measurement_concept_id_3025315,measurement_concept_id_3025634,measurement_concept_id_3027018,measurement_concept_id_3027114,measurement_concept_id_3027801,measurement_concept_id_3027946,measurement_concept_id_3028167,measurement_concept_id_3033891,measurement_concept_id_3036277,measurement_concept_id_3038288,measurement_concept_id_3038297,measurement_concept_id_3041623,measurement_concept_id_3042194,measurement_concept_id_3042596,measurement_concept_id_3044254,measurement_concept_id_3044938,measurement_concept_id_3045716,measurement_concept_id_3046279,measurement_concept_id_4196147,measurement_concept_id_40765161,measurement_concept_id_42870366
0,0,1,1,1,1,1,0,1,1,1,0,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1
1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1
2,2,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,1
3,3,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1
4,4,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [145]:
from os import listdir
from os.path import isfile, join
mypath = "."
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['person_Obs_proc_vis_drug_measu_devi_encounter.csv',
 'observation_period.csv',
 'drug_exposure.csv',
 'goldstandard_person_Obs_proc_vis_drug_measu_devi_encounter.csv',
 'Untitled.ipynb',
 'goldstandard.csv',
 'device_exposure.csv',
 'location.csv',
 'measurement.csv',
 'condition_occurrence.csv',
 'analysis_3.ipynb',
 'drug_era.csv',
 'visit_occurrence.csv',
 'data.pkl',
 'person.csv',
 'analysis_2.ipynb',
 'observation.csv',
 'procedure_occurrence.csv',
 'condition_era.csv']

In [146]:
#del twocol['Company_Name']
#unity = procedure_occurrence_grouped.merge(visit_occurrence_grouped, how = 'left', on="visit_occurrence_id")
#unity.info()

In [147]:
procedure_occurrence_visit_occurrence = pd.merge(processed_visit_final,
                                        processed_procedure_grouped,#[['person_id', 'status']],
                                        on='person_id',
                                        how = 'left'        )

In [148]:
procedure_occurrence_visit_occurrence.shape

(9119, 997)

In [149]:
#procedure_occurrence_visit_occurrence

In [150]:

Obs_proc_vis = pd.merge(processed_observation_grouped,
                        procedure_occurrence_visit_occurrence,#[['person_id', 'status']],
                        on='person_id',
                        how = 'left')


Obs_proc_vis.shape

(9132, 1004)

In [151]:

Obs_proc_vis_drug = pd.merge(drug_quantity_refill_dose,
                        Obs_proc_vis,#[['person_id', 'status']],
                        on='person_id',
                        how = 'left')


Obs_proc_vis_drug.shape




(9134, 5361)

In [152]:

Obs_proc_vis_drug_measu = pd.merge(Obs_proc_vis_drug,
                            measurement_data_grouped,#[['person_id', 'status']],
                            on='person_id',
                            how = 'left')


Obs_proc_vis_drug_measu.shape



(9134, 5430)

In [153]:
device_exposure_subset.shape

(884, 2)

In [154]:
device_exposure_subset.device_concept_id.value_counts()

45768197    884
Name: device_concept_id, dtype: int64

In [155]:

#Obs_proc_vis_drug_measu_devi = pd.merge(Obs_proc_vis_drug_measu,
 #                           device_exposure_subset,#[['person_id', 'status']],
  #                          on='person_id',
   #                         how = 'left')


#Obs_proc_vis_drug_measu_devi.shape




#device_exposure_grouped

In [156]:

Obs_proc_vis_drug_measu_devi_encounter = pd.merge(Obs_proc_vis_drug_measu,
                                                    condition_occurrence_count_data,#[['person_id', 'status']],
                                                    on='person_id',
                                                    how = 'left')


Obs_proc_vis_drug_measu_devi_encounter.shape


(9134, 6294)

In [157]:
#Obs_proc_vis_drug_measu_devi_encountercolumnslumns()

In [158]:
person_Obs_proc_vis_drug_measu_devi_encounter = pd.merge(person_ethnicity,
                                                    Obs_proc_vis_drug_measu_devi_encounter,#[['person_id', 'status']],
                                                    on='person_id',
                                                    how = 'left')


person_Obs_proc_vis_drug_measu_devi_encounter.shape




(9134, 6304)

In [159]:
#goldstandard_person_Obs_proc_vis_drug_measu_devi_encounter = pd.merge(person_Obs_proc_vis_drug_measu_devi_encounter,
#                                                    goldstandard,#[['person_id', 'status']],
#                                                    on='person_id',
#                                                    how = 'left')


#goldstandard_person_Obs_proc_vis_drug_measu_devi_encounter.info()


#person_Obs_proc_vis_drug_measu_devi_encounter.head()

In [160]:
person_Obs_proc_vis_drug_measu_devi_encounter.status.value_counts()

0.0    8206
1.0     928
Name: status, dtype: int64

In [161]:
#goldstandard_person_Obs_proc_vis_drug_measu_devi_encounter.columns

In [162]:
person_Obs_proc_vis_drug_measu_devi_encounter.shape

(9134, 6304)

In [163]:
observation_period.columns

Index(['observation_period_id', 'person_id', 'observation_period_start_date',
       'observation_period_end_date', 'period_type_concept_id'],
      dtype='object')

In [167]:
#person_Obs_proc_vis_drug_measu_devi_encounter.to_pickle("./mydata.pkl")

person_Obs_proc_vis_drug_measu_devi_encounter.status.value_counts()
#o g oi

0.0    8206
1.0     928
Name: status, dtype: int64

In [166]:
#person_Obs_proc_vis_drug_measu_devi_encounter.to_csv('mydata.csv')  

# Split Dataset at Patient Level